In [94]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

In [2]:
# Load the CSV file
data = pd.read_csv('userstory.csv', encoding='ISO-8859-1')
cdata = data.copy()

In [3]:
cdata.columns = cdata.columns.str.strip()
print("Column Names:", cdata.columns)

Column Names: Index(['User Stories', 'Description',
       'Domain [Healthcare, Ecommerce, Finance, Education, Entertainment, Media]',
       'Software Application [frontent, backend, mobile, desktop, cloud, gaming]',
       'Platform [AndroidTV,  ATV, H5TV, iOS, Android,Web, Jio STB, Jio Cinema, Jio ChargeIT, Celia, All Platforms, SDK, JioTV, Stage, Spike, Playstore, Fanverse, CTV, VOOT]'],
      dtype='object')


In [4]:
# Drop rows where 'Domain', 'Software', and 'Platform' are all null
cleaned_data = cdata.dropna(subset=['Domain [Healthcare, Ecommerce, Finance, Education, Entertainment, Media]', 'Software Application [frontent, backend, mobile, desktop, cloud, gaming]', 'Platform [AndroidTV,  ATV, H5TV, iOS, Android,Web, Jio STB, Jio Cinema, Jio ChargeIT, Celia, All Platforms, SDK, JioTV, Stage, Spike, Playstore, Fanverse, CTV, VOOT]'], how='all')

# Check the cleaned data
print(cleaned_data.head())

                                        User Stories  \
0           Pass layoutCohort to mentioned P13N APIs   
1  Optimisation and Fixes in BYA Service and Cons...   
2     Make Who's Watching Screen Title Config Driven   
3                    Making Profile Switching Easier   
4                 Launch for Trending trays via A/B    

                                         Description  \
0  *Context* : P13N services needs to make scylla...   
1  # Add LD flags for state to language mapping f...   
2  New config to be added on Launch Darkly - {{wh...   
3  Helping Users Understand the concept of Multi ...   
4  We are building Trending recommendations for o...   

  Domain [Healthcare, Ecommerce, Finance, Education, Entertainment, Media]  \
0                                                NaN                         
1                                                NaN                         
2                                              Media                         
3             

In [5]:
cleaned_data.columns = cleaned_data.columns.str.strip()
print("Column Names:", cleaned_data.columns)

Column Names: Index(['User Stories', 'Description',
       'Domain [Healthcare, Ecommerce, Finance, Education, Entertainment, Media]',
       'Software Application [frontent, backend, mobile, desktop, cloud, gaming]',
       'Platform [AndroidTV,  ATV, H5TV, iOS, Android,Web, Jio STB, Jio Cinema, Jio ChargeIT, Celia, All Platforms, SDK, JioTV, Stage, Spike, Playstore, Fanverse, CTV, VOOT]'],
      dtype='object')


In [6]:
# Get the number of rows before cleaning
initial_rows = data.shape[0]

# Get the number of rows after cleaning
cleaned_rows = cleaned_data.shape[0]

# Print the number of rows
print(f"Initial number of rows: {initial_rows}")
print(f"Number of rows after cleaning: {cleaned_rows}")

Initial number of rows: 3410
Number of rows after cleaning: 1469


In [7]:
cleaned_data.isnull().sum()

User Stories                                                                                                                                                               0
Description                                                                                                                                                              224
Domain [Healthcare, Ecommerce, Finance, Education, Entertainment, Media]                                                                                                 252
Software Application [frontent, backend, mobile, desktop, cloud, gaming]                                                                                                  55
Platform [AndroidTV,  ATV, H5TV, iOS, Android,Web, Jio STB, Jio Cinema, Jio ChargeIT, Celia, All Platforms, SDK, JioTV, Stage, Spike, Playstore, Fanverse, CTV, VOOT]    205
dtype: int64

In [8]:
y1 = cleaned_data['Domain [Healthcare, Ecommerce, Finance, Education, Entertainment, Media]']

In [9]:
y2 = cleaned_data['Software Application [frontent, backend, mobile, desktop, cloud, gaming]']

In [10]:
y3 = cleaned_data['Platform [AndroidTV,  ATV, H5TV, iOS, Android,Web, Jio STB, Jio Cinema, Jio ChargeIT, Celia, All Platforms, SDK, JioTV, Stage, Spike, Playstore, Fanverse, CTV, VOOT]']

In [11]:
y1.unique()

array([nan, 'Media', 'Entertainment', 'Finance', 'Education',
       'E-commerce'], dtype=object)

In [12]:
y2.unique()

array(['backend', 'frontend', 'cloud', 'mobile', nan, 'gaming'],
      dtype=object)

In [13]:
y3.unique()

array([nan, 'Jio Cinema', 'Web', 'Playback', 'Stage', 'Android', 'iOS',
       'MixPanel', 'AndroidTV', 'CQC', 'Celia', 'All Platforms', 'Spike',
       'All platforms', 'Playstore', 'Fanverse', 'CTV', 'ATV', 'Jio STB',
       'JioTV', 'H5TV', 'Jio ChargeIT', 'SDK', 'AppleTV', 'VOOT'],
      dtype=object)

In [14]:
y1.value_counts()

Domain [Healthcare, Ecommerce, Finance, Education, Entertainment, Media]
Media            978
E-commerce        86
Entertainment     52
Finance           51
Education         50
Name: count, dtype: int64

In [15]:
y2.value_counts()

Software Application [frontent, backend, mobile, desktop, cloud, gaming]
backend     528
mobile      397
frontend    339
cloud       110
gaming       40
Name: count, dtype: int64

In [16]:
y3.value_counts()

Platform [AndroidTV,  ATV, H5TV, iOS, Android,Web, Jio STB, Jio Cinema, Jio ChargeIT, Celia, All Platforms, SDK, JioTV, Stage, Spike, Playstore, Fanverse, CTV, VOOT]
Android          324
iOS              225
Web              178
Jio Cinema       157
VOOT             120
Playback         102
ATV               31
Jio ChargeIT      27
All platforms     24
AndroidTV         20
Fanverse          18
Playstore          7
Spike              6
Celia              5
Jio STB            4
H5TV               3
All Platforms      2
JioTV              2
CQC                2
SDK                2
AppleTV            2
MixPanel           1
CTV                1
Stage              1
Name: count, dtype: int64

In [17]:
# Merging specific platforms into a new category 'TV'
tv_platforms = ['AndroidTV', 'ATV', 'H5TV', 'Jio STB', 'JioTV', 'CTV', 'AppleTV']

# Create a copy of y3
y3_merged = y3.copy()

# Replace the specified platforms with 'TV'
y3_merged = y3_merged.replace(tv_platforms, 'TV')

# Check the new value counts after merging
platform_counts = y3_merged.value_counts()
print(platform_counts)

Platform [AndroidTV,  ATV, H5TV, iOS, Android,Web, Jio STB, Jio Cinema, Jio ChargeIT, Celia, All Platforms, SDK, JioTV, Stage, Spike, Playstore, Fanverse, CTV, VOOT]
Android          324
iOS              225
Web              178
Jio Cinema       157
VOOT             120
Playback         102
TV                63
Jio ChargeIT      27
All platforms     24
Fanverse          18
Playstore          7
Spike              6
Celia              5
CQC                2
All Platforms      2
SDK                2
Stage              1
MixPanel           1
Name: count, dtype: int64


In [18]:
# Calculate the value counts of the merged platform data
platform_counts = y3_merged.value_counts()

# Identify platforms with counts of 10 or more
platforms_to_keep = platform_counts[platform_counts >= 10].index.tolist()

# Filter the y3_merged to keep only rows with the specified platforms
y3_cleaned = y3_merged[y3_merged.isin(platforms_to_keep)]

# Display the value counts of the cleaned data
cleaned_platform_counts = y3_cleaned.value_counts()
print(cleaned_platform_counts)

Platform [AndroidTV,  ATV, H5TV, iOS, Android,Web, Jio STB, Jio Cinema, Jio ChargeIT, Celia, All Platforms, SDK, JioTV, Stage, Spike, Playstore, Fanverse, CTV, VOOT]
Android          324
iOS              225
Web              178
Jio Cinema       157
VOOT             120
Playback         102
TV                63
Jio ChargeIT      27
All platforms     24
Fanverse          18
Name: count, dtype: int64


In [19]:
y1.value_counts()

Domain [Healthcare, Ecommerce, Finance, Education, Entertainment, Media]
Media            978
E-commerce        86
Entertainment     52
Finance           51
Education         50
Name: count, dtype: int64

In [20]:
y2.value_counts()

Software Application [frontent, backend, mobile, desktop, cloud, gaming]
backend     528
mobile      397
frontend    339
cloud       110
gaming       40
Name: count, dtype: int64

In [21]:
y3_cleaned.value_counts()

Platform [AndroidTV,  ATV, H5TV, iOS, Android,Web, Jio STB, Jio Cinema, Jio ChargeIT, Celia, All Platforms, SDK, JioTV, Stage, Spike, Playstore, Fanverse, CTV, VOOT]
Android          324
iOS              225
Web              178
Jio Cinema       157
VOOT             120
Playback         102
TV                63
Jio ChargeIT      27
All platforms     24
Fanverse          18
Name: count, dtype: int64

In [22]:
print("Column Names:", cleaned_data.columns)

Column Names: Index(['User Stories', 'Description',
       'Domain [Healthcare, Ecommerce, Finance, Education, Entertainment, Media]',
       'Software Application [frontent, backend, mobile, desktop, cloud, gaming]',
       'Platform [AndroidTV,  ATV, H5TV, iOS, Android,Web, Jio STB, Jio Cinema, Jio ChargeIT, Celia, All Platforms, SDK, JioTV, Stage, Spike, Playstore, Fanverse, CTV, VOOT]'],
      dtype='object')


In [23]:
print(cleaned_data.head)

<bound method NDFrame.head of                                            User Stories  \
0              Pass layoutCohort to mentioned P13N APIs   
1     Optimisation and Fixes in BYA Service and Cons...   
2        Make Who's Watching Screen Title Config Driven   
3                       Making Profile Switching Easier   
4                    Launch for Trending trays via A/B    
...                                                 ...   
3405  As a user, I want to access character bios and...   
3406  As a gamer, I want to create and share my own ...   
3407  As a player, I want to receive feedback on my ...   
3408  As a user, I want to have access to multilingu...   
3409  As a gamer, I want to utilize augmented realit...   

                                            Description  \
0     *Context* : P13N services needs to make scylla...   
1     # Add LD flags for state to language mapping f...   
2     New config to be added on Launch Darkly - {{wh...   
3     Helping Users Under

In [24]:
# Combine story and description into one column for vectorization
cleaned_data['combined_text'] = cleaned_data['User Stories'] + " " + cleaned_data['Description']

/var/folders/c_/kmq275hx6xvflqnbfp5t2vk80000gp/T/ipykernel_98352/494726911.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  cleaned_data['combined_text'] = cleaned_data['User Stories'] + " " + cleaned_data['Description']


In [64]:
pip install -U sentence-transformers

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 255.2/255.2 kB 5.8 MB/s eta 0:00:00a 0:00:01
Note: you may need to restart the kernel to use updated packages.


In [25]:
"""# Fill missing values in the 'combined_text' column with an empty string
cleaned_data['combined_text'].fillna('', inplace=True)
# Initialize the TF-IDF vectorizer
tfidf = TfidfVectorizer(max_features=5000)  # You can adjust the number of features

# Fit and transform the combined text data
X = tfidf.fit_transform(cleaned_data['combined_text']).toarray()

/var/folders/c_/kmq275hx6xvflqnbfp5t2vk80000gp/T/ipykernel_98352/6898585.py:2: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  cleaned_data['combined_text'].fillna('', inplace=True)
/var/folders/c_/kmq275hx6xvflqnbfp5t2vk80000gp/T/ipykernel_98352/6898585.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  cleaned_data['combined_text'].fillna('',

In [98]:
from sentence_transformers import SentenceTransformer
model = SentenceTransformer('paraphrase-MiniLM-L6-v2')

# Fill missing values in the 'combined_text' column with an empty string
cleaned_data['combined_text'].fillna('', inplace=True)

# Sentences are encoded by calling model.encode()
X = model.encode(cleaned_data['combined_text'])

/opt/anaconda3/lib/python3.12/site-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


In [68]:
# Initialize LabelEncoder
le = LabelEncoder()

# Encode y1 (Domain), y2 (Software), and y3_cleaned (Platform)
y1_encoded = le.fit_transform(y1)
y2_encoded = le.fit_transform(y2)
y3_cleaned_encoded = le.fit_transform(y3_cleaned)

print(f"y1 (Domain) Encoded: {y1_encoded}")
print(f"y2 (Software) Encoded: {y2_encoded}")
print(f"y3_cleaned (Platform) Encoded: {y3_cleaned_encoded}")

y1 (Domain) Encoded: [5 5 4 ... 2 2 2]
y2 (Software) Encoded: [0 0 2 ... 3 3 3]
y3_cleaned (Platform) Encoded: [4 8 5 ... 1 9 1]


In [70]:
# Initialize the TF-IDF Vectorizer
tfidf = TfidfVectorizer(max_features=5000)  # Adjust number of features as needed

# Create the feature matrix X from combined text
# Make sure you're using cleaned_data and not any previous filtered data
X = tfidf.fit_transform(cleaned_data['combined_text']).toarray()

# Create the target DataFrame
target_data = pd.DataFrame({
    'y1': y1,
    'y2': y2,
    'y3_cleaned': y3_cleaned
})

# Concatenate features and targets
data_with_targets = pd.DataFrame(X)
data_with_targets['y1'] = target_data['y1']
data_with_targets['y2'] = target_data['y2']
data_with_targets['y3_cleaned'] = target_data['y3_cleaned']

# Drop rows with any missing values in the target columns
data_with_targets = data_with_targets.dropna()

#Recreate the feature matrix X and the target variables after alignment
X_cleaned = data_with_targets.iloc[:, :-3].values  # All columns except last three (targets)
y1_encoded = data_with_targets['y1'].values
y2_encoded = data_with_targets['y2'].values
y3_cleaned_encoded = data_with_targets['y3_cleaned'].values

# Split for Domain (y1)
X_train, X_test, y1_train, y1_test = train_test_split(X_cleaned, y1_encoded, test_size=0.2, random_state=42)

# Split for Software (y2)
X_train_y2, X_test_y2, y2_train, y2_test = train_test_split(X_cleaned, y2_encoded, test_size=0.2, random_state=42)

# Split for Platform (y3_cleaned)
X_train_y3, X_test_y3, y3_train, y3_test = train_test_split(X_cleaned, y3_cleaned_encoded, test_size=0.2, random_state=42)

# Display shapes to verify everything is correct
print(f"X_train shape: {X_train.shape}, y1_train shape: {y1_train.shape}")
print(f"X_train_y2 shape: {X_train_y2.shape}, y2_train shape: {y2_train.shape}")
print(f"X_train_y3 shape: {X_train_y3.shape}, y3_train shape: {y3_train.shape}")


X_train shape: (556, 5000), y1_train shape: (556,)
X_train_y2 shape: (556, 5000), y2_train shape: (556,)
X_train_y3 shape: (556, 5000), y3_train shape: (556,)


In [96]:
# Define models to evaluate
models = {
    'Random Forest': RandomForestClassifier(random_state=42),
    'Gradient Boosting': GradientBoostingClassifier(random_state=42),
    'Support Vector Classifier': SVC(random_state=42),
    'Logistic Regression': LogisticRegression(max_iter=1000, random_state=42)
}

# Function to train and evaluate models
def evaluate_models(X_train, X_test, y_train, y_test):
    results = {}
    for model_name, model in models.items():
        model.fit(X_train, y_train)
        y_pred = model.predict(X_test)
        accuracy = accuracy_score(y_test, y_pred)
        results[model_name] = accuracy
        print(f'{model_name} Accuracy: {accuracy:.4f}')
    return results

# Train and evaluate for each target variable
print("\n--- Evaluating models for y1 (Domain) ---")
results_y1 = evaluate_models(X_train, X_test, y1_train, y1_test)

print("\n--- Evaluating models for y2 (Software) ---")
results_y2 = evaluate_models(X_train_y2, X_test_y2, y2_train, y2_test)

print("\n--- Evaluating models for y3 (Platform) ---")
results_y3 = evaluate_models(X_train_y3, X_test_y3, y3_train, y3_test)


--- Evaluating models for y1 (Domain) ---
Random Forest Accuracy: 0.9429
Gradient Boosting Accuracy: 0.9357
Support Vector Classifier Accuracy: 0.9429
Logistic Regression Accuracy: 0.9429

--- Evaluating models for y2 (Software) ---
Random Forest Accuracy: 0.3929
Gradient Boosting Accuracy: 0.3357
Support Vector Classifier Accuracy: 0.3571
Logistic Regression Accuracy: 0.3643

--- Evaluating models for y3 (Platform) ---
Random Forest Accuracy: 0.2714
Gradient Boosting Accuracy: 0.2643
Support Vector Classifier Accuracy: 0.2714
Logistic Regression Accuracy: 0.2714


In [ ]:
"""# Print the sizes of the datasets
print("Sizes of the datasets:")
print(f"X_train shape: {X_train.shape}, y1_train shape: {y1_train.shape}")
print(f"X_test shape: {X_test.shape}, y1_test shape: {y1_test.shape}")

print(f"\nX_train_y2 shape: {X_train_y2.shape}, y2_train shape: {y2_train.shape}")
print(f"X_test_y2 shape: {X_test_y2.shape}, y2_test shape: {y2_test.shape}")

print(f"\nX_train_y3 shape: {X_train_y3.shape}, y3_train shape: {y3_train.shape}")
print(f"X_test_y3 shape: {X_test_y3.shape}, y3_test shape: {y3_test.shape}")


In [ ]:
#cleaned_data.to_csv('cleaned_data.csv', index=False)

In [32]:
pip install tf-keras

Note: you may need to restart the kernel to use updated packages.


In [34]:
pip install tqdm

Note: you may need to restart the kernel to use updated packages.


KeyError: "None of [Index([            nan,             nan,         'Media',             nan,\n                   nan,         'Media',             nan,         'Media',\n               'Media',             nan,\n       ...\n       'Entertainment', 'Entertainment', 'Entertainment', 'Entertainment',\n       'Entertainment', 'Entertainment', 'Entertainment', 'Entertainment',\n       'Entertainment', 'Entertainment'],\n      dtype='object', length=1469)] are in the [columns]"

KeyError: "None of [Index([            nan,             nan,         'Media',             nan,\n                   nan,         'Media',             nan,         'Media',\n               'Media',             nan,\n       ...\n       'Entertainment', 'Entertainment', 'Entertainment', 'Entertainment',\n       'Entertainment', 'Entertainment', 'Entertainment', 'Entertainment',\n       'Entertainment', 'Entertainment'],\n      dtype='object', length=1469)] are in the [columns]"

In [108]:
streaming_values = ["Jio Cinema", "VOOT", "Playback", "ATV", "All Platforms", "Android TV", "Jio STB", 
                    "H5TV", "JioTV", "AppleTV", "CTV", "All platforms", "AndroidTV"]

# Replace the specified values with 'streaming services'
cleaned_data[platform_col] = cleaned_data[platform_col].replace(streaming_values, 'streaming services')
cleaned_data[platform_col].value_counts()

Platform [AndroidTV,  ATV, H5TV, iOS, Android,Web, Jio STB, Jio Cinema, Jio ChargeIT, Celia, All Platforms, SDK, JioTV, Stage, Spike, Playstore, Fanverse, CTV, VOOT]
streaming services    468
Android               324
iOS                   225
Web                   178
Jio ChargeIT           27
Fanverse               18
Playstore               7
Spike                   6
Celia                   5
CQC                     2
SDK                     2
Stage                   1
MixPanel                1
Name: count, dtype: int64

In [116]:
cleaned_data = cleaned_data[cleaned_data[platform_col].map(cleaned_data[platform_col].value_counts()) >= 30]
cleaned_data[platform_col].value_counts()

Platform [AndroidTV,  ATV, H5TV, iOS, Android,Web, Jio STB, Jio Cinema, Jio ChargeIT, Celia, All Platforms, SDK, JioTV, Stage, Spike, Playstore, Fanverse, CTV, VOOT]
streaming services    468
Android               324
iOS                   225
Web                   178
Name: count, dtype: int64

In [124]:
cleaned_data[software_col].value_counts()

Software Application [frontent, backend, mobile, desktop, cloud, gaming]
mobile      395
backend     378
frontend    242
cloud       101
gaming       38
Name: count, dtype: int64

In [126]:
cleaned_data[domain_col].value_counts()

Domain [Healthcare, Ecommerce, Finance, Education, Entertainment, Media]
Media            897
E-commerce        57
Finance           44
Entertainment     40
Education         40
Name: count, dtype: int64

In [48]:
# Attempt 1 with all values greater than 10 and combined TV value
"""
Accuracy for Domain: 0.5867937372362151
Accuracy for Software: 0.4567733151803948
Accuracy for Platform: 0.4874063989108237
"""

import pandas as pd
from transformers import pipeline
from tqdm import tqdm  # For progress tracking

classifier = pipeline("zero-shot-classification", model="MoritzLaurer/mDeBERTa-v3-base-mnli-xnli", device=0)

def classify_input(query, candidate_labels, n=3):
    sequence_to_classify = f"The query is {query}"
    result = classifier(sequence_to_classify, candidate_labels)
    result_class = result["labels"][:n]
    return result_class
    
domain_col = 'Domain [Healthcare, Ecommerce, Finance, Education, Entertainment, Media]'
software_col = 'Software Application [frontent, backend, mobile, desktop, cloud, gaming]'
platform_col = 'Platform [AndroidTV,  ATV, H5TV, iOS, Android,Web, Jio STB, Jio Cinema, Jio ChargeIT, Celia, All Platforms, SDK, JioTV, Stage, Spike, Playstore, Fanverse, CTV, VOOT]'

# Prepare the candidate labels (unique values in each target column)
candidate_labels_y1 = cleaned_data[domain_col].dropna().unique().tolist()
candidate_labels_y2 = cleaned_data[software_col].dropna().unique().tolist()
candidate_labels_y3 = cleaned_data[platform_col].dropna().unique().tolist()

# Create empty columns to store the classification results
    cleaned_data['predicted_Domain'] = None
    cleaned_data['predicted_Software'] = None
    cleaned_data['predicted_Platform'] = None
    
    # Progress bar setup for classifying combined_text
    for idx, row in tqdm(cleaned_data.iterrows(), total=len(cleaned_data), desc="Classifying combined_text"):
        query = row['combined_text']
        
        # Classify for each target (Domain, Software, Platform)
    predicted_y1 = classify_input(query, candidate_labels_y1)
    predicted_y2 = classify_input(query, candidate_labels_y2)
    predicted_y3 = classify_input(query, candidate_labels_y3)

    # Store the results back into the DataFrame
    cleaned_data.at[idx, 'predicted_Domain'] = predicted_y1[0] if predicted_y1 else None
    cleaned_data.at[idx, 'predicted_Software'] = predicted_y2[0] if predicted_y2 else None
    cleaned_data.at[idx, 'predicted_Platform'] = predicted_y3[0] if predicted_y3 else None

# Check accuracy for each target class by comparing with the original data
accuracy_y1 = (cleaned_data['predicted_Domain'] == cleaned_data[domain_col]).mean()
accuracy_y2 = (cleaned_data['predicted_Software'] == cleaned_data[software_col]).mean()
accuracy_y3 = (cleaned_data['predicted_Platform'] == cleaned_data[platform_col]).mean()

print(f"Accuracy for Domain: {accuracy_y1}")
print(f"Accuracy for Software: {accuracy_y2}")
print(f"Accuracy for Platform: {accuracy_y3}")

# Save the classified data with predictions to a new CSV file
cleaned_data.to_csv('classified_cleaned_data.csv', index=False)


Classifying combined_text: 100%|██████████| 1469/1469 [1:47:01<00:00,  4.37s/it]

Accuracy for Domain: 0.5867937372362151
Accuracy for Software: 0.4567733151803948
Accuracy for Platform: 0.4874063989108237


In [122]:
#Attempt 2 with combining all TV and streaming platforms into Streaming services, along with dropping everything except for Web, iOS and Android.
"""
Accuracy for Domain: 0.6585774058577406
Accuracy for Software: 0.46778242677824267
Accuracy for Platform: 0.5774058577405857
"""
import pandas as pd
from transformers import pipeline
from tqdm import tqdm  # For progress tracking

classifier = pipeline("zero-shot-classification", model="MoritzLaurer/mDeBERTa-v3-base-mnli-xnli", device=0)

def classify_input(query, candidate_labels, n=3):
    sequence_to_classify = f"The query is {query}"
    result = classifier(sequence_to_classify, candidate_labels)
    result_class = result["labels"][:n]
    return result_class
    
domain_col = 'Domain [Healthcare, Ecommerce, Finance, Education, Entertainment, Media]'
software_col = 'Software Application [frontent, backend, mobile, desktop, cloud, gaming]'
platform_col = 'Platform [AndroidTV,  ATV, H5TV, iOS, Android,Web, Jio STB, Jio Cinema, Jio ChargeIT, Celia, All Platforms, SDK, JioTV, Stage, Spike, Playstore, Fanverse, CTV, VOOT]'

# Prepare the candidate labels (unique values in each target column)
candidate_labels_y1 = cleaned_data[domain_col].dropna().unique().tolist()
candidate_labels_y2 = cleaned_data[software_col].dropna().unique().tolist()
candidate_labels_y3 = cleaned_data[platform_col].dropna().unique().tolist()

# Create empty columns to store the classification results
cleaned_data['predicted_Domain'] = None
cleaned_data['predicted_Software'] = None
cleaned_data['predicted_Platform'] = None
    
# Progress bar setup for classifying combined_text
for idx, row in tqdm(cleaned_data.iterrows(), total=len(cleaned_data), desc="Classifying combined_text"):
    query = row['combined_text']
        
    # Classify for each target (Domain, Software, Platform)
    predicted_y1 = classify_input(query, candidate_labels_y1)
    predicted_y2 = classify_input(query, candidate_labels_y2)
    predicted_y3 = classify_input(query, candidate_labels_y3)

    # Store the results back into the DataFrame
    cleaned_data.at[idx, 'predicted_Domain'] = predicted_y1[0] if predicted_y1 else None
    cleaned_data.at[idx, 'predicted_Software'] = predicted_y2[0] if predicted_y2 else None
    cleaned_data.at[idx, 'predicted_Platform'] = predicted_y3[0] if predicted_y3 else None

# Check accuracy for each target class by comparing with the original data
accuracy_y1 = (cleaned_data['predicted_Domain'] == cleaned_data[domain_col]).mean()
accuracy_y2 = (cleaned_data['predicted_Software'] == cleaned_data[software_col]).mean()
accuracy_y3 = (cleaned_data['predicted_Platform'] == cleaned_data[platform_col]).mean()

print(f"Accuracy for Domain: {accuracy_y1}")
print(f"Accuracy for Software: {accuracy_y2}")
print(f"Accuracy for Platform: {accuracy_y3}")

# Save the classified data with predictions to a new CSV file
cleaned_data.to_csv('classified_cleaned_data.csv', index=False)

Classifying combined_text: 100%|██████████| 1195/1195 [1:00:59<00:00,  3.06s/it]


Accuracy for Domain: 0.6585774058577406
Accuracy for Software: 0.46778242677824267
Accuracy for Platform: 0.5774058577405857


In [56]:
"""import pandas as pd
from transformers import pipeline
from tqdm import tqdm  # For progress tracking

# Load cleaned data
cleaned_data = pd.read_csv('cleaned_data.csv')

# Drop rows where 'combined_text' is NaN
cleaned_data.dropna(subset=['combined_text'], inplace=True)

# Initialize the zero-shot classification pipeline
classifier = pipeline("zero-shot-classification", model="MoritzLaurer/mDeBERTa-v3-base-mnli-xnli", device=0)

def classify_input(batch, candidate_labels):
    # Use the pipeline to classify the entire batch
    results = classifier(batch, candidate_labels)
    return results

# Define columns for classification
domain_col = 'Domain [Healthcare, Ecommerce, Finance, Education, Entertainment, Media]'
software_col = 'Software Application [frontent, backend, mobile, desktop, cloud, gaming]'
platform_col = 'Platform [AndroidTV,  ATV, H5TV, iOS, Android,Web, Jio STB, Jio Cinema, Jio ChargeIT, Celia, All Platforms, SDK, JioTV, Stage, Spike, Playstore, Fanverse, CTV, VOOT]'

# Prepare the candidate labels (unique values in each target column)
candidate_labels_y1 = cleaned_data[domain_col].dropna().unique().tolist()
candidate_labels_y2 = cleaned_data[software_col].dropna().unique().tolist()
candidate_labels_y3 = cleaned_data[platform_col].dropna().unique().tolist()

# Create empty columns to store the classification results
cleaned_data['predicted_Domain'] = None
cleaned_data['predicted_Software'] = None
cleaned_data['predicted_Platform'] = None

# Define the batch size
batch_size = 8

# Progress bar setup for classifying combined_text
for start_idx in tqdm(range(0, len(cleaned_data), batch_size), desc="Batch Classifying combined_text"):
    end_idx = min(start_idx + batch_size, len(cleaned_data))
    batch = cleaned_data['combined_text'].iloc[start_idx:end_idx].tolist()
    
    # Classify the batch for each target (Domain, Software, Platform)
    predictions_y1 = classify_input(batch, candidate_labels_y1)
    predictions_y2 = classify_input(batch, candidate_labels_y2)
    predictions_y3 = classify_input(batch, candidate_labels_y3)
    
    # Store the predictions back into the DataFrame
    for i in range(len(batch)):
        # Safeguard against potential missing predictions
        cleaned_data.at[start_idx + i, 'predicted_Domain'] = predictions_y1[i]["labels"][0] if predictions_y1 else None
        cleaned_data.at[start_idx + i, 'predicted_Software'] = predictions_y2[i]["labels"][0] if predictions_y2 else None
        cleaned_data.at[start_idx + i, 'predicted_Platform'] = predictions_y3[i]["labels"][0] if predictions_y3 else None

# Check accuracy for each target class by comparing with the original data
accuracy_y1 = (cleaned_data['predicted_Domain'] == cleaned_data[domain_col]).mean()
accuracy_y2 = (cleaned_data['predicted_Software'] == cleaned_data[software_col]).mean()
accuracy_y3 = (cleaned_data['predicted_Platform'] == cleaned_data[platform_col]).mean()

print(f"Accuracy for Domain: {accuracy_y1}")
print(f"Accuracy for Software: {accuracy_y2}")
print(f"Accuracy for Platform: {accuracy_y3}")

# Save the classified data with predictions to a new CSV file
cleaned_data.to_csv('classified_cleaned_data.csv', index=False)


Batch Classifying combined_text:   1%|        | 2/156 [02:29<3:12:14, 74.90s/it]


KeyboardInterrupt: 

In [ ]:
"""import pandas as pd
from transformers import pipeline
from tqdm import tqdm  # For progress tracking
from multiprocessing import Pool, cpu_count

# Load the cleaned data
cleaned_data = pd.read_csv('cleaned_data.csv')

# Initialize the classifier pipeline
classifier = pipeline("zero-shot-classification", model="MoritzLaurer/mDeBERTa-v3-base-mnli-xnli", device=0)

# Define the function for classifying a single row
def classify_row(row):
    query = row['combined_text']
    # Classify for each target (Domain, Software, Platform)
    predicted_y1 = classify_input(query, candidate_labels_y1)
    predicted_y2 = classify_input(query, candidate_labels_y2)
    predicted_y3 = classify_input(query, candidate_labels_y3)
    
    return {
        'predicted_Domain': predicted_y1[0] if predicted_y1 else None,
        'predicted_Software': predicted_y2[0] if predicted_y2 else None,
        'predicted_Platform': predicted_y3[0] if predicted_y3 else None
    }

def classify_input(query, candidate_labels, n=3):
    sequence_to_classify = f"The query is {query}"
    result = classifier(sequence_to_classify, candidate_labels)
    result_class = result["labels"][:n]
    return result_class

# Define the columns to classify
domain_col = 'Domain [Healthcare, Ecommerce, Finance, Education, Entertainment, Media]'
software_col = 'Software Application [frontent, backend, mobile, desktop, cloud, gaming]'
platform_col = 'Platform [AndroidTV,  ATV, H5TV, iOS, Android,Web, Jio STB, Jio Cinema, Jio ChargeIT, Celia, All Platforms, SDK, JioTV, Stage, Spike, Playstore, Fanverse, CTV, VOOT]'

# Prepare the candidate labels (unique values in each target column)
candidate_labels_y1 = cleaned_data[domain_col].dropna().unique().tolist()
candidate_labels_y2 = cleaned_data[software_col].dropna().unique().tolist()
candidate_labels_y3 = cleaned_data[platform_col].dropna().unique().tolist()

# Create a list to hold the rows of predictions
predictions = []

# Use multiprocessing to classify the data
if __name__ == '__main__':
    with Pool(processes=cpu_count()) as pool:
        results = list(tqdm(pool.imap(classify_row, [row for _, row in cleaned_data.iterrows()]), total=len(cleaned_data), desc="Classifying combined_text"))
    
    # Populate the predictions back into the DataFrame
    for result in results:
        predictions.append(result)

    # Expand predictions into the DataFrame
    cleaned_data['predicted_Domain'] = [result['predicted_Domain'] for result in predictions]
    cleaned_data['predicted_Software'] = [result['predicted_Software'] for result in predictions]
    cleaned_data['predicted_Platform'] = [result['predicted_Platform'] for result in predictions]

# Check accuracy for each target class by comparing with the original data
accuracy_y1 = (cleaned_data['predicted_Domain'] == cleaned_data[domain_col]).mean()
accuracy_y2 = (cleaned_data['predicted_Software'] == cleaned_data[software_col]).mean()
accuracy_y3 = (cleaned_data['predicted_Platform'] == cleaned_data[platform_col]).mean()

print(f"Accuracy for Domain: {accuracy_y1}")
print(f"Accuracy for Software: {accuracy_y2}")
print(f"Accuracy for Platform: {accuracy_y3}")

# Save the classified data with predictions to a new CSV file
cleaned_data.to_csv('classified_cleaned_data.csv', index=False)

if __name__ == '__main__':
    main()


In [174]:
import pandas as pd

# Load the cleaned_data DataFrame
cleaned_data = pd.read_csv('cleaned_data.csv')

# Column name for Domain
domain_col = 'Domain [Healthcare, Ecommerce, Finance, Education, Entertainment, Media]'

# List of the five categories
categories = ['Media', 'E-commerce', 'Entertainment', 'Finance', 'Education']

# Create an empty DataFrame to store the 250 rows
sampled_data = pd.DataFrame()

# For each category, sample 50 rows and append them to the new DataFrame
for category in categories:
    category_sample = cleaned_data[cleaned_data[domain_col] == category].sample(n=50, random_state=42)
    sampled_data = pd.concat([sampled_data, category_sample])

# Reset the index of the new DataFrame
sampled_data = sampled_data.reset_index(drop=True)

# Print the value counts of the Domain column in the new DataFrame
print("Value counts in the new DataFrame:")
print(sampled_data[domain_col].value_counts())

# Save the sampled data to a new CSV file if needed
sampled_data.to_csv('sampled_cleaned_data.csv', index=False)

Value counts in the new DataFrame:
Domain [Healthcare, Ecommerce, Finance, Education, Entertainment, Media]
Media            50
E-commerce       50
Entertainment    50
Finance          50
Education        50
Name: count, dtype: int64


In [232]:
sampled_data['Platform [AndroidTV,  ATV, H5TV, iOS, Android,Web, Jio STB, Jio Cinema, Jio ChargeIT, Celia, All Platforms, SDK, JioTV, Stage, Spike, Playstore, Fanverse, CTV, VOOT]'
].value_counts()

Platform [AndroidTV,  ATV, H5TV, iOS, Android,Web, Jio STB, Jio Cinema, Jio ChargeIT, Celia, All Platforms, SDK, JioTV, Stage, Spike, Playstore, Fanverse, CTV, VOOT]
Android               60
Web                   50
iOS                   48
streaming services    39
Jio ChargeIT          12
Name: count, dtype: int64

In [138]:
sampled_data[software_col].value_counts()

Software Application [frontent, backend, mobile, desktop, cloud, gaming]
cloud       82
backend     69
gaming      38
frontend    33
mobile      26
Name: count, dtype: int64

In [144]:
# Assuming 'cleaned_data' is your DataFrame and 'Platform' is the column with platform values
streaming_values = ["Jio Cinema", "VOOT", "Playback", "ATV", "All Platforms", "AndroidTV", "Jio STB", 
                    "H5TV", "JioTV", "AppleTV", "CTV", "All platforms"]

# Replace the specified values with 'streaming services'
sampled_data[platform_col] = sampled_data[platform_col].replace(streaming_values, 'streaming services')

# Verify the changes
print(sampled_data[platform_col].value_counts())

Platform [AndroidTV,  ATV, H5TV, iOS, Android,Web, Jio STB, Jio Cinema, Jio ChargeIT, Celia, All Platforms, SDK, JioTV, Stage, Spike, Playstore, Fanverse, CTV, VOOT]
Android               60
Web                   50
iOS                   48
streaming services    39
Jio ChargeIT          12
Fanverse               9
Playstore              5
Celia                  5
Name: count, dtype: int64


In [150]:
# Check the number of rows in the sampled_data DataFrame
num_rows = sampled_data.shape[0]
print(f"Number of rows in the sampled_data: {num_rows}")

Number of rows in the sampled_data: 250


In [152]:
#Attempt 3 With total of 250 rows with 50 from each of the Domains
"""
Accuracy for Domain: 0.656
Accuracy for Software: 0.472
Accuracy for Platform: 0.596
"""
import pandas as pd
from transformers import pipeline
from tqdm import tqdm  # For progress tracking

classifier = pipeline("zero-shot-classification", model="MoritzLaurer/mDeBERTa-v3-base-mnli-xnli", device=0)

def classify_input(query, candidate_labels, n=3):
    sequence_to_classify = f"The query is {query}"
    result = classifier(sequence_to_classify, candidate_labels)
    result_class = result["labels"][:n]
    return result_class
    
domain_col = 'Domain [Healthcare, Ecommerce, Finance, Education, Entertainment, Media]'
software_col = 'Software Application [frontent, backend, mobile, desktop, cloud, gaming]'
platform_col = 'Platform [AndroidTV,  ATV, H5TV, iOS, Android,Web, Jio STB, Jio Cinema, Jio ChargeIT, Celia, All Platforms, SDK, JioTV, Stage, Spike, Playstore, Fanverse, CTV, VOOT]'

# Prepare the candidate labels (unique values in each target column)
candidate_labels_y1 = sampled_data[domain_col].dropna().unique().tolist()
candidate_labels_y2 = sampled_data[software_col].dropna().unique().tolist()
candidate_labels_y3 = sampled_data[platform_col].dropna().unique().tolist()

# Create empty columns to store the classification results
sampled_data['predicted_Domain'] = None
sampled_data['predicted_Software'] = None
sampled_data['predicted_Platform'] = None
    
# Progress bar setup for classifying combined_text
for idx, row in tqdm(sampled_data.iterrows(), total=len(sampled_data), desc="Classifying combined_text"):
    query = row['combined_text']
        
    # Classify for each target (Domain, Software, Platform)
    predicted_y1 = classify_input(query, candidate_labels_y1)
    predicted_y2 = classify_input(query, candidate_labels_y2)
    predicted_y3 = classify_input(query, candidate_labels_y3)

    # Store the results back into the DataFrame
    sampled_data.at[idx, 'predicted_Domain'] = predicted_y1[0] if predicted_y1 else None
    sampled_data.at[idx, 'predicted_Software'] = predicted_y2[0] if predicted_y2 else None
    sampled_data.at[idx, 'predicted_Platform'] = predicted_y3[0] if predicted_y3 else None

# Check accuracy for each target class by comparing with the original data
accuracy_y1 = (sampled_data['predicted_Domain'] == sampled_data[domain_col]).mean()
accuracy_y2 = (sampled_data['predicted_Software'] == sampled_data[software_col]).mean()
accuracy_y3 = (sampled_data['predicted_Platform'] == sampled_data[platform_col]).mean()

print(f"Accuracy for Domain: {accuracy_y1}")
print(f"Accuracy for Software: {accuracy_y2}")
print(f"Accuracy for Platform: {accuracy_y3}")

# Save the classified data with predictions to a new CSV file
#d_data.to_csv('classified_cleaned_data.csv', index=False)

Classifying combined_text: 100%|██████████████| 250/250 [12:25<00:00,  2.98s/it]

Accuracy for Domain: 0.656
Accuracy for Software: 0.472
Accuracy for Platform: 0.596


In [154]:
sampled_data.to_csv('sampled_data.csv', index=False)

In [172]:
sampled_data.columns[4]

'Platform [AndroidTV,  ATV, H5TV, iOS, Android,Web, Jio STB, Jio Cinema, Jio ChargeIT, Celia, All Platforms, SDK, JioTV, Stage, Spike, Playstore, Fanverse, CTV, VOOT]'

In [160]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split

# Preprocessing step: Fill NaN values in combined_text column with an empty string
sampled_data['combined_text'] = sampled_data['combined_text'].fillna('')

# Step 1: Extract features using TF-IDF
tfidf_vectorizer = TfidfVectorizer(max_features=1000)
X = tfidf_vectorizer.fit_transform(sampled_data['combined_text'])

# Step 2: Split data for each target (Domain, Software, Platform)
domain_col = 'Domain [Healthcare, Ecommerce, Finance, Education, Entertainment, Media]'
software_col = 'Software Application [frontent, backend, mobile, desktop, cloud, gaming]'
platform_col = 'Platform [AndroidTV, ATV, H5TV, iOS, Android,Web, Jio STB, Jio Cinema, Jio ChargeIT, Celia, All Platforms, SDK, JioTV, Stage, Spike, Playstore, Fanverse, CTV, VOOT]'

# Split for Domain
X_train_y1, X_test_y1, y_train_y1, y_test_y1 = train_test_split(X, sampled_data[domain_col], test_size=0.2, random_state=42)

# Split for Software
X_train_y2, X_test_y2, y_train_y2, y_test_y2 = train_test_split(X, sampled_data[software_col], test_size=0.2, random_state=42)

# Split for Platform
X_train_y3, X_test_y3, y_train_y3, y_test_y3 = train_test_split(X, sampled_data[platform_col], test_size=0.2, random_state=42)

KeyError: 'Platform [AndroidTV, ATV, H5TV, iOS, Android,Web, Jio STB, Jio Cinema, Jio ChargeIT, Celia, All Platforms, SDK, JioTV, Stage, Spike, Playstore, Fanverse, CTV, VOOT]'

In [178]:
import pandas as pd

# Get all columns in the sampled_data DataFrame
columns = sampled_data.columns.tolist()

# Print the list of columns
print("Columns in sampled_data:")
print(columns)


Columns in sampled_data:
['User Stories', 'Description', 'Domain [Healthcare, Ecommerce, Finance, Education, Entertainment, Media]', 'Software Application [frontent, backend, mobile, desktop, cloud, gaming]', 'Platform [AndroidTV,  ATV, H5TV, iOS, Android,Web, Jio STB, Jio Cinema, Jio ChargeIT, Celia, All Platforms, SDK, JioTV, Stage, Spike, Playstore, Fanverse, CTV, VOOT]', 'combined_text']


In [186]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report
from tqdm import tqdm  # For progress tracking

# Assuming `sampled_data` is your DataFrame containing the required columns.

# Preprocessing: Fill NaN values and convert target columns to string
sampled_data['combined_text'] = sampled_data['combined_text'].fillna('').astype(str)

# Prepare target variables
y_domain = sampled_data['Domain [Healthcare, Ecommerce, Finance, Education, Entertainment, Media]'].fillna('').astype(str)
y_software = sampled_data['Software Application [frontent, backend, mobile, desktop, cloud, gaming]'].fillna('').astype(str)
y_platform = sampled_data['Platform [AndroidTV,  ATV, H5TV, iOS, Android,Web, Jio STB, Jio Cinema, Jio ChargeIT, Celia, All Platforms, SDK, JioTV, Stage, Spike, Playstore, Fanverse, CTV, VOOT]'].fillna('').astype(str)

# Preprocessing and feature extraction
vectorizer = TfidfVectorizer()
X = vectorizer.fit_transform(sampled_data['combined_text'])

# Split the data into training and testing sets
X_train, X_test, y_train_domain, y_test_domain = train_test_split(X, y_domain, test_size=0.2, random_state=42)
X_train, X_test, y_train_software, y_test_software = train_test_split(X, y_software, test_size=0.2, random_state=42)
X_train, X_test, y_train_platform, y_test_platform = train_test_split(X, y_platform, test_size=0.2, random_state=42)

# Define the model and hyperparameter grid
model = RandomForestClassifier(random_state=42)
param_grid = {
    'n_estimators': [50, 100],
    'max_depth': [None, 10, 20],
    'min_samples_split': [2, 5],
}

# Grid search for hyperparameter tuning
grid_search = GridSearchCV(model, param_grid, cv=5, scoring='f1_macro')

# Fit model for each target variable
for target, y_train, y_test in [('Domain', y_train_domain, y_test_domain),
                                 ('Software', y_train_software, y_test_software),
                                 ('Platform', y_train_platform, y_test_platform)]:
    grid_search.fit(X_train, y_train)
    y_pred = grid_search.predict(X_test)

    # Evaluate model
    print(f"Results for {target}:")
    print(classification_report(y_test, y_pred))

# To see the best parameters found by GridSearchCV for each target
print("Best parameters found for Domain:", grid_search.best_params_)


Results for Domain:
               precision    recall  f1-score   support

   E-commerce       0.33      0.86      0.48         7
    Education       1.00      1.00      1.00         9
Entertainment       1.00      0.70      0.82        10
      Finance       1.00      0.64      0.78        11
        Media       0.78      0.54      0.64        13

     accuracy                           0.72        50
    macro avg       0.82      0.75      0.74        50
 weighted avg       0.85      0.72      0.75        50



/opt/anaconda3/lib/python3.12/site-packages/sklearn/model_selection/_split.py:737: UserWarning: The least populated class in y has only 2 members, which is less than n_splits=5.
  warnings.warn(


Results for Software:
              precision    recall  f1-score   support

     backend       0.42      1.00      0.59        10
       cloud       1.00      1.00      1.00        16
    frontend       0.67      0.20      0.31        10
      gaming       1.00      0.88      0.93         8
      mobile       0.00      0.00      0.00         6

    accuracy                           0.70        50
   macro avg       0.62      0.61      0.57        50
weighted avg       0.70      0.70      0.65        50



/opt/anaconda3/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/opt/anaconda3/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/opt/anaconda3/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/opt/anaconda3/lib/python3.12/site-pa

Results for Platform:
                    precision    recall  f1-score   support

                         0.00      0.00      0.00         2
           Android       0.50      0.60      0.55        10
             Celia       0.00      0.00      0.00         3
          Fanverse       0.00      0.00      0.00         2
      Jio ChargeIT       0.17      0.50      0.25         2
               Web       0.91      0.83      0.87        12
               iOS       0.71      0.42      0.53        12
streaming services       0.33      0.57      0.42         7

          accuracy                           0.52        50
         macro avg       0.33      0.37      0.33        50
      weighted avg       0.54      0.52      0.51        50

Best parameters found for Domain: {'max_depth': None, 'min_samples_split': 2, 'n_estimators': 100}


/opt/anaconda3/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/opt/anaconda3/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/opt/anaconda3/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


In [190]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report
from imblearn.over_sampling import SMOTE  # For handling imbalanced classes

# Assuming `sampled_data` is your DataFrame containing the required columns.

# Preprocessing: Fill NaN values and convert target columns to string
sampled_data['combined_text'] = sampled_data['combined_text'].fillna('').astype(str)

# Prepare target variables
y_domain = sampled_data['Domain [Healthcare, Ecommerce, Finance, Education, Entertainment, Media]'].fillna('').astype(str)
y_software = sampled_data['Software Application [frontent, backend, mobile, desktop, cloud, gaming]'].fillna('').astype(str)
y_platform = sampled_data['Platform [AndroidTV,  ATV, H5TV, iOS, Android,Web, Jio STB, Jio Cinema, Jio ChargeIT, Celia, All Platforms, SDK, JioTV, Stage, Spike, Playstore, Fanverse, CTV, VOOT]'].fillna('').astype(str)
# Preprocessing and feature extraction
vectorizer = TfidfVectorizer()
X = vectorizer.fit_transform(sampled_data['combined_text'])

# Split the data into training and testing sets
X_train, X_test, y_train_domain, y_test_domain = train_test_split(X, y_domain, test_size=0.2, random_state=42)
X_train, X_test, y_train_software, y_test_software = train_test_split(X, y_software, test_size=0.2, random_state=42)
X_train, X_test, y_train_platform, y_test_platform = train_test_split(X, y_platform, test_size=0.2, random_state=42)

# Handle class imbalance using SMOTE
smote = SMOTE(random_state=42)
X_train, y_train_domain = smote.fit_resample(X_train, y_train_domain)
X_train, y_train_software = smote.fit_resample(X_train, y_train_software)
X_train, y_train_platform = smote.fit_resample(X_train, y_train_platform)

# Define the model with class weighting
model = RandomForestClassifier(random_state=42, class_weight='balanced')
param_grid = {
    'n_estimators': [50, 100],
    'max_depth': [None, 10, 20],
    'min_samples_split': [2, 5],
}

# Grid search for hyperparameter tuning
grid_search = GridSearchCV(model, param_grid, cv=3, scoring='f1_macro')  # Reduce cv to avoid warnings

# Fit model for each target variable
for target, y_train, y_test in [('Domain', y_train_domain, y_test_domain),
                                 ('Software', y_train_software, y_test_software),
                                 ('Platform', y_train_platform, y_test_platform)]:
    grid_search.fit(X_train, y_train)
    y_pred = grid_search.predict(X_test)

    # Evaluate model
    print(f"Results for {target}:")
    print(classification_report(y_test, y_pred, zero_division=0))  # Control zero division behavior

# To see the best parameters found by GridSearchCV for each target
print("Best parameters found for Domain:", grid_search.best_params_)


ValueError: Found input variables with inconsistent numbers of samples: [215, 200]

In [198]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report
from imblearn.over_sampling import SMOTE  # For handling imbalanced classes

# Assuming `sampled_data` is your DataFrame containing the required columns.

# Preprocessing: Fill NaN values and convert target columns to string
sampled_data['combined_text'] = sampled_data['combined_text'].fillna('').astype(str)

# Prepare target variables
y_domain = sampled_data['Domain [Healthcare, Ecommerce, Finance, Education, Entertainment, Media]'].fillna('').astype(str)
y_software = sampled_data['Software Application [frontent, backend, mobile, desktop, cloud, gaming]'].fillna('').astype(str)
y_platform = sampled_data['Platform [AndroidTV,  ATV, H5TV, iOS, Android,Web, Jio STB, Jio Cinema, Jio ChargeIT, Celia, All Platforms, SDK, JioTV, Stage, Spike, Playstore, Fanverse, CTV, VOOT]'].fillna('').astype(str)

# Preprocessing and feature extraction
vectorizer = TfidfVectorizer()
X = vectorizer.fit_transform(sampled_data['combined_text'])

# Split the data into training and testing sets
X_train, X_test, y_train_domain, y_test_domain = train_test_split(X, y_domain, test_size=0.2, random_state=42)
X_train, X_test, y_train_software, y_test_software = train_test_split(X, y_software, test_size=0.2, random_state=42)
X_train, X_test, y_train_platform, y_test_platform = train_test_split(X, y_platform, test_size=0.2, random_state=42)

# Define SMOTE with fewer neighbors
smote = SMOTE(random_state=42, k_neighbors=1)  # Reduce k_neighbors to 1 for safety

# Function to apply SMOTE and check the number of samples
def apply_smote(X_train, y_train):
    if len(y_train.value_counts()) < 2:  # Check if there are fewer than 2 classes
        print("Not enough classes for SMOTE.")
        return X_train, y_train  # Return original if not enough classes
    return smote.fit_resample(X_train, y_train)

# Resample training set for each target variable
X_train_domain, y_train_domain = smote.fit_resample(X_train, y_train_domain)
X_train_software, y_train_software = smote.fit_resample(X_train, y_train_software)
X_train_platform, y_train_platform = smote.fit_resample(X_train, y_train_platform)

# Define the model with class weighting
model = RandomForestClassifier(random_state=42, class_weight='balanced')
param_grid = {
    'n_estimators': [50, 100],
    'max_depth': [None, 10, 20],
    'min_samples_split': [2, 5],
}

# Grid search for hyperparameter tuning
grid_search = GridSearchCV(model, param_grid, cv=3, scoring='f1_macro')  # Reduce cv to avoid warnings

# Fit model for each target variable
for target, X_train, y_train, y_test in [('Domain', X_train_domain, y_train_domain, y_test_domain),
                                          ('Software', X_train_software, y_train_software, y_test_software),
                                          ('Platform', X_train_platform, y_train_platform, y_test_platform)]:
    grid_search.fit(X_train, y_train)
    y_pred = grid_search.predict(X_test)

    # Evaluate model
    print(f"Results for {target}:")
    print(classification_report(y_test, y_pred, zero_division=0))  # Control zero division behavior

# To see the best parameters found by GridSearchCV for each target
print("Best parameters found for Domain:", grid_search.best_params_)


Results for Domain:
               precision    recall  f1-score   support

   E-commerce       0.42      0.71      0.53         7
    Education       1.00      1.00      1.00         9
Entertainment       1.00      0.70      0.82        10
      Finance       1.00      0.64      0.78        11
        Media       0.67      0.77      0.71        13

     accuracy                           0.76        50
    macro avg       0.82      0.76      0.77        50
 weighted avg       0.83      0.76      0.78        50

Results for Software:
              precision    recall  f1-score   support

     backend       0.43      0.90      0.58        10
       cloud       1.00      1.00      1.00        16
    frontend       0.60      0.30      0.40        10
      gaming       1.00      0.88      0.93         8
      mobile       0.00      0.00      0.00         6

    accuracy                           0.70        50
   macro avg       0.61      0.61      0.58        50
weighted avg       0.69   

In [214]:
print(sampled_data.columns)

Index(['User Stories', 'Description',
       'Domain [Healthcare, Ecommerce, Finance, Education, Entertainment, Media]',
       'Software Application [frontent, backend, mobile, desktop, cloud, gaming]',
       'Platform [AndroidTV,  ATV, H5TV, iOS, Android,Web, Jio STB, Jio Cinema, Jio ChargeIT, Celia, All Platforms, SDK, JioTV, Stage, Spike, Playstore, Fanverse, CTV, VOOT]',
       'combined_text', 'predicted_Domain', 'predicted_Software',
       'predicted_Platform'],
      dtype='object')


In [218]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report
from imblearn.over_sampling import SMOTE  # For handling imbalanced classes

# Define the categories to drop
categories_to_drop = ['mobile', 'Celia', 'Fanverse']

# Filter the dataset to drop the specified categories from 'Platform'
sampled_data = sampled_data[~sampled_data['Platform [AndroidTV,  ATV, H5TV, iOS, Android,Web, Jio STB, Jio Cinema, Jio ChargeIT, Celia, All Platforms, SDK, JioTV, Stage, Spike, Playstore, Fanverse, CTV, VOOT]'].isin(categories_to_drop)]

# Preprocessing: Fill NaN values and convert target columns to string
sampled_data['combined_text'] = sampled_data['combined_text'].fillna('').astype(str)

# Prepare target variables
y_domain = sampled_data['Domain [Healthcare, Ecommerce, Finance, Education, Entertainment, Media]'].fillna('').astype(str)
y_software = sampled_data['Software Application [frontent, backend, mobile, desktop, cloud, gaming]'].fillna('').astype(str)
y_platform = sampled_data['Platform [AndroidTV,  ATV, H5TV, iOS, Android,Web, Jio STB, Jio Cinema, Jio ChargeIT, Celia, All Platforms, SDK, JioTV, Stage, Spike, Playstore, Fanverse, CTV, VOOT]'].fillna('').astype(str)

# Preprocessing and feature extraction
vectorizer = TfidfVectorizer()
X = vectorizer.fit_transform(sampled_data['combined_text'])

# Split the data into training and testing sets
X_train, X_test, y_train_domain, y_test_domain = train_test_split(X, y_domain, test_size=0.2, random_state=42)
X_train, X_test, y_train_software, y_test_software = train_test_split(X, y_software, test_size=0.2, random_state=42)
X_train, X_test, y_train_platform, y_test_platform = train_test_split(X, y_platform, test_size=0.2, random_state=42)

# Define SMOTE with fewer neighbors
smote = SMOTE(random_state=42, k_neighbors=1)  # Reduce k_neighbors to 1 for safety

# Function to apply SMOTE and check the number of samples
def apply_smote(X_train, y_train):
    if len(y_train.value_counts()) < 2:  # Check if there are fewer than 2 classes
        print("Not enough classes for SMOTE.")
        return X_train, y_train  # Return original if not enough classes
    return smote.fit_resample(X_train, y_train)

# Resample training set for each target variable
X_train_domain, y_train_domain = apply_smote(X_train, y_train_domain)
X_train_software, y_train_software = apply_smote(X_train, y_train_software)
X_train_platform, y_train_platform = apply_smote(X_train, y_train_platform)

# Define the model with class weighting
model = RandomForestClassifier(random_state=42, class_weight='balanced')
param_grid = {
    'n_estimators': [50, 100],
    'max_depth': [None, 10, 20],
    'min_samples_split': [2, 5],
}

# Grid search for hyperparameter tuning
grid_search = GridSearchCV(model, param_grid, cv=3, scoring='f1_macro')  # Reduce cv to avoid warnings

# Fit model for each target variable
for target, X_train, y_train, y_test in [('Domain', X_train_domain, y_train_domain, y_test_domain),
                                          ('Software', X_train_software, y_train_software, y_test_software),
                                          ('Platform', X_train_platform, y_train_platform, y_test_platform)]:
    grid_search.fit(X_train, y_train)
    y_pred = grid_search.predict(X_test)

    # Evaluate model
    print(f"Results for {target}:")
    print(classification_report(y_test, y_pred, zero_division=0))  # Control zero division behavior

# To see the best parameters found by GridSearchCV for each target
print("Best parameters found for Domain:", grid_search.best_params_)

/var/folders/c_/kmq275hx6xvflqnbfp5t2vk80000gp/T/ipykernel_98352/3845267513.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  sampled_data['combined_text'] = sampled_data['combined_text'].fillna('').astype(str)


Results for Domain:
               precision    recall  f1-score   support

   E-commerce       0.73      0.89      0.80         9
    Education       1.00      0.91      0.95        11
Entertainment       1.00      1.00      1.00         7
      Finance       0.90      0.82      0.86        11
        Media       0.70      0.70      0.70        10

     accuracy                           0.85        48
    macro avg       0.87      0.86      0.86        48
 weighted avg       0.86      0.85      0.86        48

Results for Software:
              precision    recall  f1-score   support

     backend       0.60      0.75      0.67        12
       cloud       0.94      1.00      0.97        17
    frontend       1.00      0.29      0.44         7
      gaming       1.00      0.86      0.92         7
      mobile       0.43      0.60      0.50         5

    accuracy                           0.77        48
   macro avg       0.79      0.70      0.70        48
weighted avg       0.82   

In [220]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report
from imblearn.over_sampling import SMOTE  # For handling imbalanced classes

# Define the categories to drop
categories_to_drop = ['Playstore', '']

# Filter the dataset to drop the specified categories from 'Platform'
sampled_data = sampled_data[~sampled_data['Platform [AndroidTV,  ATV, H5TV, iOS, Android,Web, Jio STB, Jio Cinema, Jio ChargeIT, Celia, All Platforms, SDK, JioTV, Stage, Spike, Playstore, Fanverse, CTV, VOOT]'].isin(categories_to_drop)]

# Preprocessing: Fill NaN values and convert target columns to string
sampled_data['combined_text'] = sampled_data['combined_text'].fillna('').astype(str)

# Prepare target variables
y_domain = sampled_data['Domain [Healthcare, Ecommerce, Finance, Education, Entertainment, Media]'].fillna('').astype(str)
y_software = sampled_data['Software Application [frontent, backend, mobile, desktop, cloud, gaming]'].fillna('').astype(str)
y_platform = sampled_data['Platform [AndroidTV,  ATV, H5TV, iOS, Android,Web, Jio STB, Jio Cinema, Jio ChargeIT, Celia, All Platforms, SDK, JioTV, Stage, Spike, Playstore, Fanverse, CTV, VOOT]'].fillna('').astype(str)

# Preprocessing and feature extraction
vectorizer = TfidfVectorizer()
X = vectorizer.fit_transform(sampled_data['combined_text'])

# Split the data into training and testing sets
X_train, X_test, y_train_domain, y_test_domain = train_test_split(X, y_domain, test_size=0.2, random_state=42)
X_train, X_test, y_train_software, y_test_software = train_test_split(X, y_software, test_size=0.2, random_state=42)
X_train, X_test, y_train_platform, y_test_platform = train_test_split(X, y_platform, test_size=0.2, random_state=42)

# Define SMOTE with fewer neighbors
smote = SMOTE(random_state=42, k_neighbors=1)  # Reduce k_neighbors to 1 for safety

# Function to apply SMOTE and check the number of samples
def apply_smote(X_train, y_train):
    if len(y_train.value_counts()) < 2:  # Check if there are fewer than 2 classes
        print("Not enough classes for SMOTE.")
        return X_train, y_train  # Return original if not enough classes
    return smote.fit_resample(X_train, y_train)

# Resample training set for each target variable
X_train_domain, y_train_domain = apply_smote(X_train, y_train_domain)
X_train_software, y_train_software = apply_smote(X_train, y_train_software)
X_train_platform, y_train_platform = apply_smote(X_train, y_train_platform)

# Define the model with class weighting
model = RandomForestClassifier(random_state=42, class_weight='balanced')
param_grid = {
    'n_estimators': [50, 100],
    'max_depth': [None, 10, 20],
    'min_samples_split': [2, 5],
}

# Grid search for hyperparameter tuning
grid_search = GridSearchCV(model, param_grid, cv=3, scoring='f1_macro')  # Reduce cv to avoid warnings

# Fit model for each target variable
for target, X_train, y_train, y_test in [('Domain', X_train_domain, y_train_domain, y_test_domain),
                                          ('Software', X_train_software, y_train_software, y_test_software),
                                          ('Platform', X_train_platform, y_train_platform, y_test_platform)]:
    grid_search.fit(X_train, y_train)
    y_pred = grid_search.predict(X_test)

    # Evaluate model
    print(f"Results for {target}:")
    print(classification_report(y_test, y_pred, zero_division=0))  # Control zero division behavior

# To see the best parameters found by GridSearchCV for each target
print("Best parameters found for Domain:", grid_search.best_params_)

Results for Domain:
               precision    recall  f1-score   support

   E-commerce       0.50      0.86      0.63         7
    Education       1.00      0.90      0.95        10
Entertainment       1.00      0.55      0.71        11
      Finance       1.00      1.00      1.00        11
        Media       0.56      0.62      0.59         8

     accuracy                           0.79        47
    macro avg       0.81      0.79      0.77        47
 weighted avg       0.85      0.79      0.80        47

Results for Software:
              precision    recall  f1-score   support

     backend       0.50      0.89      0.64         9
       cloud       1.00      1.00      1.00        18
    frontend       0.40      0.29      0.33         7
      gaming       1.00      0.67      0.80         9
      mobile       0.50      0.25      0.33         4

    accuracy                           0.74        47
   macro avg       0.68      0.62      0.62        47
weighted avg       0.77   

In [224]:
print(sampled_data.shape)

(231, 9)


In [228]:
from sentence_transformers import SentenceTransformer
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
import pandas as pd

cleaned_data['combined_text'] = cleaned_data['combined_text'].fillna('')

# Initialize the sentence transformer model
model = SentenceTransformer('all-MiniLM-L6-v2')  # You can choose another model if needed

# Create the feature matrix X from combined text using sentence-transformer
# The embeddings are stored in X
X = model.encode(cleaned_data['combined_text'].tolist())

# Create the target DataFrame
target_data = pd.DataFrame({
    'y1': y1,
    'y2': y2,
    'y3_cleaned': y3_cleaned
})

# Concatenate features and targets
data_with_targets = pd.DataFrame(X)
data_with_targets['y1'] = target_data['y1']
data_with_targets['y2'] = target_data['y2']
data_with_targets['y3_cleaned'] = target_data['y3_cleaned']

# Drop rows with any missing values in the target columns
data_with_targets = data_with_targets.dropna()

# Recreate the feature matrix X and the target variables after alignment
X_cleaned = data_with_targets.iloc[:, :-3].values  # All columns except last three (targets)
y1_encoded = data_with_targets['y1'].values
y2_encoded = data_with_targets['y2'].values
y3_cleaned_encoded = data_with_targets['y3_cleaned'].values

# Split for Domain (y1)
X_train, X_test, y1_train, y1_test = train_test_split(X_cleaned, y1_encoded, test_size=0.2, random_state=42)

# Split for Software (y2)
X_train_y2, X_test_y2, y2_train, y2_test = train_test_split(X_cleaned, y2_encoded, test_size=0.2, random_state=42)

# Split for Platform (y3_cleaned)
X_train_y3, X_test_y3, y3_train, y3_test = train_test_split(X_cleaned, y3_cleaned_encoded, test_size=0.2, random_state=42)

# Display shapes to verify everything is correct
print(f"X_train shape: {X_train.shape}, y1_train shape: {y1_train.shape}")
print(f"X_train_y2 shape: {X_train_y2.shape}, y2_train shape: {y2_train.shape}")
print(f"X_train_y3 shape: {X_train_y3.shape}, y3_train shape: {y3_train.shape}")

# Define models to evaluate
models = {
    'Random Forest': RandomForestClassifier(random_state=42),
    'Gradient Boosting': GradientBoostingClassifier(random_state=42),
    'Support Vector Classifier': SVC(random_state=42),
    'Logistic Regression': LogisticRegression(max_iter=1000, random_state=42)
}

# Function to train and evaluate models
def evaluate_models(X_train, X_test, y_train, y_test):
    results = {}
    for model_name, model in models.items():
        model.fit(X_train, y_train)
        y_pred = model.predict(X_test)
        accuracy = accuracy_score(y_test, y_pred)
        results[model_name] = accuracy
        print(f'{model_name} Accuracy: {accuracy:.4f}')
    return results

# Train and evaluate for each target variable
print("\n--- Evaluating models for y1 (Domain) ---")
results_y1 = evaluate_models(X_train, X_test, y1_train, y1_test)

print("\n--- Evaluating models for y2 (Software) ---")
results_y2 = evaluate_models(X_train_y2, X_test_y2, y2_train, y2_test)

print("\n--- Evaluating models for y3 (Platform) ---")
results_y3 = evaluate_models(X_train_y3, X_test_y3, y3_train, y3_test)


/opt/anaconda3/lib/python3.12/site-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


X_train shape: (556, 384), y1_train shape: (556,)
X_train_y2 shape: (556, 384), y2_train shape: (556,)
X_train_y3 shape: (556, 384), y3_train shape: (556,)

--- Evaluating models for y1 (Domain) ---
Random Forest Accuracy: 0.9429
Gradient Boosting Accuracy: 0.9214
Support Vector Classifier Accuracy: 0.9429
Logistic Regression Accuracy: 0.9429

--- Evaluating models for y2 (Software) ---
Random Forest Accuracy: 0.3643
Gradient Boosting Accuracy: 0.3214
Support Vector Classifier Accuracy: 0.3571
Logistic Regression Accuracy: 0.3714

--- Evaluating models for y3 (Platform) ---
Random Forest Accuracy: 0.2643
Gradient Boosting Accuracy: 0.2500
Support Vector Classifier Accuracy: 0.2643
Logistic Regression Accuracy: 0.2214


In [236]:
file_path = 'sampled_cleaned_data.csv'  # Replace with the actual path to your CSV file
data = pd.read_csv(file_path)

# Assuming 'cleaned_data' is your DataFrame and 'Platform' is the column with platform values
streaming_values = ["Jio Cinema", "VOOT", "Playback", "ATV", "All Platforms", "AndroidTV", "Jio STB", 
                    "H5TV", "JioTV", "AppleTV", "CTV", "All platforms"]

# Replace the specified values with 'streaming services'
sampled_data['Platform [AndroidTV,  ATV, H5TV, iOS, Android,Web, Jio STB, Jio Cinema, Jio ChargeIT, Celia, All Platforms, SDK, JioTV, Stage, Spike, Playstore, Fanverse, CTV, VOOT]'] = sampled_data['Platform [AndroidTV,  ATV, H5TV, iOS, Android,Web, Jio STB, Jio Cinema, Jio ChargeIT, Celia, All Platforms, SDK, JioTV, Stage, Spike, Playstore, Fanverse, CTV, VOOT]'].replace(streaming_values, 'streaming services')

import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report
from imblearn.over_sampling import SMOTE  # For handling imbalanced classes

# Define the categories to drop
categories_to_drop = ['mobile', 'Celia', 'Fanverse']

# Filter the dataset to drop the specified categories from 'Platform'
sampled_data = sampled_data[~sampled_data['Platform [AndroidTV,  ATV, H5TV, iOS, Android,Web, Jio STB, Jio Cinema, Jio ChargeIT, Celia, All Platforms, SDK, JioTV, Stage, Spike, Playstore, Fanverse, CTV, VOOT]'].isin(categories_to_drop)]

# Preprocessing: Fill NaN values and convert target columns to string
sampled_data['combined_text'] = sampled_data['combined_text'].fillna('').astype(str)

# Prepare target variables
y_domain = sampled_data['Domain [Healthcare, Ecommerce, Finance, Education, Entertainment, Media]'].fillna('').astype(str)
y_software = sampled_data['Software Application [frontent, backend, mobile, desktop, cloud, gaming]'].fillna('').astype(str)
y_platform = sampled_data['Platform [AndroidTV,  ATV, H5TV, iOS, Android,Web, Jio STB, Jio Cinema, Jio ChargeIT, Celia, All Platforms, SDK, JioTV, Stage, Spike, Playstore, Fanverse, CTV, VOOT]'].fillna('').astype(str)

# Preprocessing and feature extraction
vectorizer = TfidfVectorizer()
X = vectorizer.fit_transform(sampled_data['combined_text'])

# Split the data into training and testing sets
X_train, X_test, y_train_domain, y_test_domain = train_test_split(X, y_domain, test_size=0.2, random_state=42)
X_train, X_test, y_train_software, y_test_software = train_test_split(X, y_software, test_size=0.2, random_state=42)
X_train, X_test, y_train_platform, y_test_platform = train_test_split(X, y_platform, test_size=0.2, random_state=42)

# Define SMOTE with fewer neighbors
smote = SMOTE(random_state=42, k_neighbors=1)  # Reduce k_neighbors to 1 for safety

# Function to apply SMOTE and check the number of samples
def apply_smote(X_train, y_train):
    if len(y_train.value_counts()) < 2:  # Check if there are fewer than 2 classes
        print("Not enough classes for SMOTE.")
        return X_train, y_train  # Return original if not enough classes
    return smote.fit_resample(X_train, y_train)

# Resample training set for each target variable
X_train_domain, y_train_domain = apply_smote(X_train, y_train_domain)
X_train_software, y_train_software = apply_smote(X_train, y_train_software)
X_train_platform, y_train_platform = apply_smote(X_train, y_train_platform)

# Define the model with class weighting
model = RandomForestClassifier(random_state=42, class_weight='balanced')
param_grid = {
    'n_estimators': [50, 100],
    'max_depth': [None, 10, 20],
    'min_samples_split': [2, 5],
}

# Grid search for hyperparameter tuning
grid_search = GridSearchCV(model, param_grid, cv=3, scoring='f1_macro')  # Reduce cv to avoid warnings

# Fit model for each target variable
for target, X_train, y_train, y_test in [('Domain', X_train_domain, y_train_domain, y_test_domain),
                                          ('Software', X_train_software, y_train_software, y_test_software),
                                          ('Platform', X_train_platform, y_train_platform, y_test_platform)]:
    grid_search.fit(X_train, y_train)
    y_pred = grid_search.predict(X_test)

    # Evaluate model
    print(f"Results for {target}:")
    print(classification_report(y_test, y_pred, zero_division=0))  # Control zero division behavior

# To see the best parameters found by GridSearchCV for each target
print("Best parameters found for Domain:", grid_search.best_params_)

Results for Domain:
               precision    recall  f1-score   support

   E-commerce       0.50      0.86      0.63         7
    Education       1.00      0.90      0.95        10
Entertainment       1.00      0.55      0.71        11
      Finance       1.00      1.00      1.00        11
        Media       0.56      0.62      0.59         8

     accuracy                           0.79        47
    macro avg       0.81      0.79      0.77        47
 weighted avg       0.85      0.79      0.80        47

Results for Software:
              precision    recall  f1-score   support

     backend       0.50      0.89      0.64         9
       cloud       1.00      1.00      1.00        18
    frontend       0.40      0.29      0.33         7
      gaming       1.00      0.67      0.80         9
      mobile       0.50      0.25      0.33         4

    accuracy                           0.74        47
   macro avg       0.68      0.62      0.62        47
weighted avg       0.77   

In [242]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report
from imblearn.over_sampling import SMOTE

# Define the categories to drop
categories_to_drop = ['mobile', 'Celia', 'Fanverse']

# Filter the dataset to drop the specified categories from 'Platform'
sampled_data = sampled_data[~sampled_data['Platform [AndroidTV,  ATV, H5TV, iOS, Android,Web, Jio STB, Jio Cinema, Jio ChargeIT, Celia, All Platforms, SDK, JioTV, Stage, Spike, Playstore, Fanverse, CTV, VOOT]'].isin(categories_to_drop)]

# Preprocessing: Fill NaN values and convert target columns to string
sampled_data['combined_text'] = sampled_data['combined_text'].fillna('').astype(str)

# Prepare target variables
y_domain = sampled_data['Domain [Healthcare, Ecommerce, Finance, Education, Entertainment, Media]'].fillna('').astype(str)
y_software = sampled_data['Software Application [frontent, backend, mobile, desktop, cloud, gaming]'].fillna('').astype(str)
y_platform = sampled_data['Platform [AndroidTV,  ATV, H5TV, iOS, Android,Web, Jio STB, Jio Cinema, Jio ChargeIT, Celia, All Platforms, SDK, JioTV, Stage, Spike, Playstore, Fanverse, CTV, VOOT]'].fillna('').astype(str)

# Preprocessing and feature extraction
vectorizer = TfidfVectorizer()
X = vectorizer.fit_transform(sampled_data['combined_text'])

# Split the data into training and testing sets
X_train, X_test, y_train_domain, y_test_domain = train_test_split(X, y_domain, test_size=0.2, random_state=42)
X_train, X_test, y_train_software, y_test_software = train_test_split(X, y_software, test_size=0.2, random_state=42)
X_train, X_test, y_train_platform, y_test_platform = train_test_split(X, y_platform, test_size=0.2, random_state=42)

# Define SMOTE with fewer neighbors
smote = SMOTE(random_state=42, k_neighbors=1)

# Function to apply SMOTE and check the number of samples
def apply_smote(X_train, y_train):
    if len(y_train.value_counts()) < 2:  # Check if there are fewer than 2 classes
        print("Not enough classes for SMOTE.")
        return X_train, y_train  # Return original if not enough classes
    return smote.fit_resample(X_train, y_train)

# Resample training set for each target variable
X_train_domain, y_train_domain = apply_smote(X_train, y_train_domain)
X_train_software, y_train_software = apply_smote(X_train, y_train_software)
X_train_platform, y_train_platform = apply_smote(X_train, y_train_platform)

# Define the model dictionary
models = {
    'Random Forest': RandomForestClassifier(random_state=42, class_weight='balanced'),
    'Gradient Boosting': GradientBoostingClassifier(random_state=42),
    'Support Vector Classifier': SVC(random_state=42, class_weight='balanced'),
    'Logistic Regression': LogisticRegression(random_state=42, max_iter=1000, class_weight='balanced')
}

# Define the parameter grid for hyperparameter tuning for each model
param_grids = {
    'Random Forest': {'n_estimators': [50, 100], 'max_depth': [None, 10, 20], 'min_samples_split': [2, 5]},
    'Gradient Boosting': {'n_estimators': [50, 100], 'max_depth': [3, 5, 10]},
    'Support Vector Classifier': {'C': [0.1, 1, 10], 'kernel': ['linear', 'rbf']},
    'Logistic Regression': {'C': [0.1, 1, 10]}
}

# Loop through the models and perform training, tuning, and evaluation
for model_name, model in models.items():
    print(f"Training and evaluating {model_name}...")

    for target, X_train, y_train, y_test in [
        ('Domain', X_train_domain, y_train_domain, y_test_domain),
        ('Software', X_train_software, y_train_software, y_test_software),
        ('Platform', X_train_platform, y_train_platform, y_test_platform)
    ]:
        print(f"\nEvaluating {target} with {model_name}")

        # Define GridSearchCV for each model
        grid_search = GridSearchCV(model, param_grids[model_name], cv=3, scoring='f1_macro')

        # Fit the model
        grid_search.fit(X_train, y_train)

        # Predict and evaluate
        y_pred = grid_search.predict(X_test)
        print(f"Results for {target} with {model_name}:")
        print(classification_report(y_test, y_pred, zero_division=0))

        # Print best parameters for each model
        print(f"Best parameters found for {target} with {model_name}: {grid_search.best_params_}")

Training and evaluating Random Forest...

Evaluating Domain with Random Forest
Results for Domain with Random Forest:
               precision    recall  f1-score   support

   E-commerce       0.50      0.86      0.63         7
    Education       1.00      0.90      0.95        10
Entertainment       1.00      0.55      0.71        11
      Finance       1.00      1.00      1.00        11
        Media       0.56      0.62      0.59         8

     accuracy                           0.79        47
    macro avg       0.81      0.79      0.77        47
 weighted avg       0.85      0.79      0.80        47

Best parameters found for Domain with Random Forest: {'max_depth': 20, 'min_samples_split': 2, 'n_estimators': 100}

Evaluating Software with Random Forest
Results for Software with Random Forest:
              precision    recall  f1-score   support

     backend       0.50      0.89      0.64         9
       cloud       1.00      1.00      1.00        18
    frontend       0.40 

In [244]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report
from imblearn.over_sampling import SMOTE

# Define the categories to drop
categories_to_drop = ['mobile', 'Celia', 'Fanverse']

# Filter the dataset to drop the specified categories from 'Platform'
sampled_data = sampled_data[~sampled_data['Platform [AndroidTV,  ATV, H5TV, iOS, Android,Web, Jio STB, Jio Cinema, Jio ChargeIT, Celia, All Platforms, SDK, JioTV, Stage, Spike, Playstore, Fanverse, CTV, VOOT]'].isin(categories_to_drop)]

# Preprocessing: Fill NaN values and convert target columns to string
sampled_data['combined_text'] = sampled_data['combined_text'].fillna('').astype(str)

# Prepare target variables
y_domain = sampled_data['Domain [Healthcare, Ecommerce, Finance, Education, Entertainment, Media]'].fillna('').astype(str)
y_software = sampled_data['Software Application [frontent, backend, mobile, desktop, cloud, gaming]'].fillna('').astype(str)
y_platform = sampled_data['Platform [AndroidTV,  ATV, H5TV, iOS, Android,Web, Jio STB, Jio Cinema, Jio ChargeIT, Celia, All Platforms, SDK, JioTV, Stage, Spike, Playstore, Fanverse, CTV, VOOT]'].fillna('').astype(str)

# Load a pre-trained sentence transformer model
model_name = 'sentence-transformers/all-MiniLM-L6-v2'  # You can change this to another model if needed
sentence_model = SentenceTransformer(model_name)

# Use the sentence transformer to encode the 'combined_text' column
X = sentence_model.encode(sampled_data['combined_text'].tolist(), show_progress_bar=True)

# Split the data into training and testing sets
X_train, X_test, y_train_domain, y_test_domain = train_test_split(X, y_domain, test_size=0.2, random_state=42)
X_train, X_test, y_train_software, y_test_software = train_test_split(X, y_software, test_size=0.2, random_state=42)
X_train, X_test, y_train_platform, y_test_platform = train_test_split(X, y_platform, test_size=0.2, random_state=42)

# Define SMOTE with fewer neighbors
smote = SMOTE(random_state=42, k_neighbors=1)

# Function to apply SMOTE and check the number of samples
def apply_smote(X_train, y_train):
    if len(y_train.value_counts()) < 2:  # Check if there are fewer than 2 classes
        print("Not enough classes for SMOTE.")
        return X_train, y_train  # Return original if not enough classes
    return smote.fit_resample(X_train, y_train)

# Resample training set for each target variable
X_train_domain, y_train_domain = apply_smote(X_train, y_train_domain)
X_train_software, y_train_software = apply_smote(X_train, y_train_software)
X_train_platform, y_train_platform = apply_smote(X_train, y_train_platform)

# Define the model dictionary
models = {
    'Random Forest': RandomForestClassifier(random_state=42, class_weight='balanced'),
    'Gradient Boosting': GradientBoostingClassifier(random_state=42),
    'Support Vector Classifier': SVC(random_state=42, class_weight='balanced'),
    'Logistic Regression': LogisticRegression(random_state=42, max_iter=1000, class_weight='balanced')
}

# Define the parameter grid for hyperparameter tuning for each model
param_grids = {
    'Random Forest': {'n_estimators': [50, 100], 'max_depth': [None, 10, 20], 'min_samples_split': [2, 5]},
    'Gradient Boosting': {'n_estimators': [50, 100], 'max_depth': [3, 5, 10]},
    'Support Vector Classifier': {'C': [0.1, 1, 10], 'kernel': ['linear', 'rbf']},
    'Logistic Regression': {'C': [0.1, 1, 10]}
}

# Loop through the models and perform training, tuning, and evaluation
for model_name, model in models.items():
    print(f"Training and evaluating {model_name}...")

    for target, X_train, y_train, y_test in [
        ('Domain', X_train_domain, y_train_domain, y_test_domain),
        ('Software', X_train_software, y_train_software, y_test_software),
        ('Platform', X_train_platform, y_train_platform, y_test_platform)
    ]:
        print(f"\nEvaluating {target} with {model_name}")

        # Define GridSearchCV for each model
        grid_search = GridSearchCV(model, param_grids[model_name], cv=3, scoring='f1_macro')

        # Fit the model
        grid_search.fit(X_train, y_train)

        # Predict and evaluate
        y_pred = grid_search.predict(X_test)
        print(f"Results for {target} with {model_name}:")
        print(classification_report(y_test, y_pred, zero_division=0))

        # Print best parameters for each model
        print(f"Best parameters found for {target} with {model_name}: {grid_search.best_params_}")

/opt/anaconda3/lib/python3.12/site-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


Batches:   0%|          | 0/8 [00:00<?, ?it/s]

Training and evaluating Random Forest...

Evaluating Domain with Random Forest
Results for Domain with Random Forest:
               precision    recall  f1-score   support

   E-commerce       0.83      0.71      0.77         7
    Education       0.90      0.90      0.90        10
Entertainment       1.00      0.64      0.78        11
      Finance       1.00      1.00      1.00        11
        Media       0.46      0.75      0.57         8

     accuracy                           0.81        47
    macro avg       0.84      0.80      0.80        47
 weighted avg       0.86      0.81      0.82        47

Best parameters found for Domain with Random Forest: {'max_depth': None, 'min_samples_split': 2, 'n_estimators': 100}

Evaluating Software with Random Forest
Results for Software with Random Forest:
              precision    recall  f1-score   support

     backend       0.58      0.78      0.67         9
       cloud       0.86      1.00      0.92        18
    frontend       0.6

In [248]:
sampled_data[domain_col].value_counts()

Domain [Healthcare, Ecommerce, Finance, Education, Entertainment, Media]
Media            50
Education        50
E-commerce       47
Finance          43
Entertainment    41
Name: count, dtype: int64

In [250]:
sampled_data[software_col].value_counts()

Software Application [frontent, backend, mobile, desktop, cloud, gaming]
cloud       82
backend     60
gaming      38
mobile      26
frontend    23
Name: count, dtype: int64

In [254]:
sampled_data['Platform [AndroidTV,  ATV, H5TV, iOS, Android,Web, Jio STB, Jio Cinema, Jio ChargeIT, Celia, All Platforms, SDK, JioTV, Stage, Spike, Playstore, Fanverse, CTV, VOOT]'].value_counts()

Platform [AndroidTV,  ATV, H5TV, iOS, Android,Web, Jio STB, Jio Cinema, Jio ChargeIT, Celia, All Platforms, SDK, JioTV, Stage, Spike, Playstore, Fanverse, CTV, VOOT]
Android               60
Web                   50
iOS                   48
streaming services    39
Jio ChargeIT          12
Name: count, dtype: int64

In [256]:
# Save sampled_data as a CSV file
sampled_data.to_csv('final_data.csv', index=False)

In [258]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report
from imblearn.over_sampling import SMOTE

# Load the final_data.csv
sampled_data = pd.read_csv('final_data.csv')

# Preprocessing: Fill NaN values and convert target columns to string
sampled_data['combined_text'] = sampled_data['combined_text'].fillna('').astype(str)

# Prepare target variables
y_domain = sampled_data['Domain [Healthcare, Ecommerce, Finance, Education, Entertainment, Media]'].fillna('').astype(str)
y_software = sampled_data['Software Application [frontent, backend, mobile, desktop, cloud, gaming]'].fillna('').astype(str)
y_platform = sampled_data['Platform [AndroidTV,  ATV, H5TV, iOS, Android,Web, Jio STB, Jio Cinema, Jio ChargeIT, Celia, All Platforms, SDK, JioTV, Stage, Spike, Playstore, Fanverse, CTV, VOOT]'].fillna('').astype(str)

# Preprocessing and feature extraction
vectorizer = TfidfVectorizer()
X = vectorizer.fit_transform(sampled_data['combined_text'])

# Split the data into training and testing sets for each target
X_train_domain, X_test_domain, y_train_domain, y_test_domain = train_test_split(X, y_domain, test_size=0.2, random_state=42)
X_train_software, X_test_software, y_train_software, y_test_software = train_test_split(X, y_software, test_size=0.2, random_state=42)
X_train_platform, X_test_platform, y_train_platform, y_test_platform = train_test_split(X, y_platform, test_size=0.2, random_state=42)

# Define SMOTE with fewer neighbors
smote = SMOTE(random_state=42, k_neighbors=1)

# Function to apply SMOTE and check the number of samples
def apply_smote(X_train, y_train):
    if len(y_train.value_counts()) < 2:  # Check if there are fewer than 2 classes
        print("Not enough classes for SMOTE.")
        return X_train, y_train  # Return original if not enough classes
    return smote.fit_resample(X_train, y_train)

# Resample training set for each target variable
X_train_domain, y_train_domain = apply_smote(X_train_domain, y_train_domain)
X_train_software, y_train_software = apply_smote(X_train_software, y_train_software)
X_train_platform, y_train_platform = apply_smote(X_train_platform, y_train_platform)

# Define the model dictionary with preferred models
models = {
    'Logistic Regression': LogisticRegression(random_state=42, max_iter=1000, class_weight='balanced'),
    'Gradient Boosting': GradientBoostingClassifier(random_state=42)
}

# Define the parameter grid for hyperparameter tuning for each model
param_grids = {
    'Logistic Regression': {'C': [0.1, 1, 10]},
    'Gradient Boosting': {'n_estimators': [50, 100], 'max_depth': [3, 5, 10]}
}

# Train Logistic Regression for Domain, Gradient Boosting for Software and Platform
for model_name, model in models.items():
    if model_name == 'Logistic Regression':
        print(f"\nEvaluating Domain with {model_name}")
        grid_search = GridSearchCV(model, param_grids[model_name], cv=3, scoring='f1_macro')
        grid_search.fit(X_train_domain, y_train_domain)
        y_pred = grid_search.predict(X_test_domain)
        print("Results for Domain:")
        print(classification_report(y_test_domain, y_pred, zero_division=0))
        print(f"Best parameters: {grid_search.best_params_}")

    elif model_name == 'Gradient Boosting':
        for target, X_train, y_train, y_test in [
            ('Software', X_train_software, y_train_software, y_test_software),
            ('Platform', X_train_platform, y_train_platform, y_test_platform)
        ]:
            print(f"\nEvaluating {target} with {model_name}")
            grid_search = GridSearchCV(model, param_grids[model_name], cv=3, scoring='f1_macro')
            grid_search.fit(X_train, y_train)
            y_pred = grid_search.predict(X_test)
            print(f"Results for {target}:")
            print(classification_report(y_test, y_pred, zero_division=0))
            print(f"Best parameters: {grid_search.best_params_}")



Evaluating Domain with Logistic Regression
Results for Domain:
               precision    recall  f1-score   support

   E-commerce       0.67      0.57      0.62         7
    Education       1.00      0.80      0.89        10
Entertainment       1.00      0.55      0.71        11
      Finance       1.00      1.00      1.00        11
        Media       0.44      0.88      0.58         8

     accuracy                           0.77        47
    macro avg       0.82      0.76      0.76        47
 weighted avg       0.85      0.77      0.78        47

Best parameters: {'C': 0.1}

Evaluating Software with Gradient Boosting


ValueError: X has 384 features, but GradientBoostingClassifier is expecting 3156 features as input.

In [270]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report
from imblearn.over_sampling import SMOTE

# Load final_data from CSV
sampled_data = pd.read_csv('final_data.csv')

# Preprocessing: Fill NaN values and convert target columns to string
sampled_data['combined_text'] = sampled_data['combined_text'].fillna('').astype(str)

# Prepare target variables
y_domain = sampled_data['Domain [Healthcare, Ecommerce, Finance, Education, Entertainment, Media]'].fillna('').astype(str)
y_software = sampled_data['Software Application [frontent, backend, mobile, desktop, cloud, gaming]'].fillna('').astype(str)
y_platform = sampled_data['Platform [AndroidTV,  ATV, H5TV, iOS, Android,Web, Jio STB, Jio Cinema, Jio ChargeIT, Celia, All Platforms, SDK, JioTV, Stage, Spike, Playstore, Fanverse, CTV, VOOT]'].fillna('').astype(str)

# Preprocessing and feature extraction
vectorizer = TfidfVectorizer()
X = vectorizer.fit_transform(sampled_data['combined_text'])

# Split the data into training and testing sets once for features and domain
X_train, X_test, y_domain_train, y_domain_test = train_test_split(X, y_domain, test_size=0.2, random_state=42)

# Split the data into training and testing sets for software and platform
_, _, y_software_train, y_software_test = train_test_split(X, y_software, test_size=0.2, random_state=42)
_, _, y_platform_train, y_platform_test = train_test_split(X, y_platform, test_size=0.2, random_state=42)

# Define SMOTE
smote = SMOTE(random_state=42, k_neighbors=1)

# Function to apply SMOTE
def apply_smote(X_train, y_train):
    if len(y_train.value_counts()) < 2:  # Check if there are fewer than 2 classes
        print("Not enough classes for SMOTE.")
        return X_train, y_train  # Return original if not enough classes
    return smote.fit_resample(X_train, y_train)

# Resample training set for each target variable
X_train_domain, y_train_domain = apply_smote(X_train, y_domain_train)
X_train_software, y_train_software = apply_smote(X_train, y_software_train)
X_train_platform, y_train_platform = apply_smote(X_train, y_platform_train)

# Define the model dictionary
models = {
    'Gradient Boosting': GradientBoostingClassifier(random_state=42),
    'Logistic Regression': LogisticRegression(random_state=42, max_iter=1000, class_weight='balanced')
}

# Define the parameter grid for hyperparameter tuning for each model
param_grids = {
    'Gradient Boosting': {'n_estimators': [50, 100], 'max_depth': [3, 5, 10]},
    'Logistic Regression': {'C': [0.1, 1, 10]}
}

# Loop through the models and perform training, tuning, and evaluation
for model_name, model in models.items():
    print(f"Training and evaluating {model_name}...")

    for target, X_train, y_train, y_test in [
        ('Domain', X_train_domain, y_train_domain, y_domain_test),
        ('Software', X_train_software, y_train_software, y_software_test),
        ('Platform', X_train_platform, y_train_platform, y_platform_test)
    ]:
        print(f"\nEvaluating {target} with {model_name}")

        # Define GridSearchCV for each model
        grid_search = GridSearchCV(model, param_grids[model_name], cv=3, scoring='f1_macro')

        # Fit the model
        grid_search.fit(X_train, y_train)

        # Predict and evaluate
        y_pred = grid_search.predict(X_test)
        print(f"Results for {target} with {model_name}:")
        print(classification_report(y_test, y_pred, zero_division=0))

        # Print best parameters for each model
        print(f"Best parameters found for {target} with {model_name}: {grid_search.best_params_}")



Training and evaluating Gradient Boosting...

Evaluating Domain with Gradient Boosting
Results for Domain with Gradient Boosting:
               precision    recall  f1-score   support

   E-commerce       0.67      0.86      0.75         7
    Education       0.77      1.00      0.87        10
Entertainment       1.00      0.55      0.71        11
      Finance       1.00      0.91      0.95        11
        Media       0.67      0.75      0.71         8

     accuracy                           0.81        47
    macro avg       0.82      0.81      0.80        47
 weighted avg       0.84      0.81      0.80        47

Best parameters found for Domain with Gradient Boosting: {'max_depth': 3, 'n_estimators': 100}

Evaluating Software with Gradient Boosting
Results for Software with Gradient Boosting:
              precision    recall  f1-score   support

     backend       0.55      0.67      0.60         9
       cloud       1.00      0.94      0.97        18
    frontend       0.43  

In [272]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report
from imblearn.over_sampling import SMOTE

# Load the final dataset
sampled_data = pd.read_csv('final_data.csv')

# Preprocessing: Fill NaN values and convert target columns to string
sampled_data['combined_text'] = sampled_data['combined_text'].fillna('').astype(str)

# Prepare target variables
y_domain = sampled_data['Domain [Healthcare, Ecommerce, Finance, Education, Entertainment, Media]'].fillna('').astype(str)
y_software = sampled_data['Software Application [frontent, backend, mobile, desktop, cloud, gaming]'].fillna('').astype(str)
y_platform = sampled_data['Platform [AndroidTV,  ATV, H5TV, iOS, Android,Web, Jio STB, Jio Cinema, Jio ChargeIT, Celia, All Platforms, SDK, JioTV, Stage, Spike, Playstore, Fanverse, CTV, VOOT]'].fillna('').astype(str)

# Preprocessing and feature extraction
vectorizer = TfidfVectorizer()
X = vectorizer.fit_transform(sampled_data['combined_text'])

# Split the data into training and testing sets
X_train, X_test, y_train_domain, y_test_domain = train_test_split(X, y_domain, test_size=0.2, random_state=42)
X_train, X_test, y_train_software, y_test_software = train_test_split(X, y_software, test_size=0.2, random_state=42)
X_train, X_test, y_train_platform, y_test_platform = train_test_split(X, y_platform, test_size=0.2, random_state=42)

# Define SMOTE with fewer neighbors
smote = SMOTE(random_state=42, k_neighbors=1)

# Function to apply SMOTE and check the number of samples
def apply_smote(X_train, y_train):
    if len(y_train.value_counts()) < 2:  # Check if there are fewer than 2 classes
        print("Not enough classes for SMOTE.")
        return X_train, y_train  # Return original if not enough classes
    return smote.fit_resample(X_train, y_train)

# Resample training set for each target variable
X_train_domain, y_train_domain = apply_smote(X_train, y_train_domain)
X_train_software, y_train_software = apply_smote(X_train, y_train_software)
X_train_platform, y_train_platform = apply_smote(X_train, y_train_platform)

# Define the model dictionary
models = {
    'Logistic Regression': LogisticRegression(random_state=42, max_iter=1000, class_weight='balanced'),  # For Domain
    'Gradient Boosting Software': GradientBoostingClassifier(random_state=42),  # For Software
    'Gradient Boosting Platform': GradientBoostingClassifier(random_state=42)  # For Platform
}

# Define the parameter grid for hyperparameter tuning for each model
param_grids = {
    'Logistic Regression': {'C': [0.1, 1, 10]},
    'Gradient Boosting Software': {'n_estimators': [50, 100], 'max_depth': [3, 5, 10]},
    'Gradient Boosting Platform': {'n_estimators': [50, 100], 'max_depth': [3, 5, 10]}
}

# Loop through the models and perform training, tuning, and evaluation
for model_name, model in models.items():
    print(f"Training and evaluating {model_name}...")

    # Determine the target variable
    if "Domain" in model_name:
        target, X_train, y_train, y_test = 'Domain', X_train_domain, y_train_domain, y_test_domain
    else:
        # For Gradient Boosting, determine the target based on model name
        if "Software" in model_name:
            target, X_train, y_train, y_test = 'Software', X_train_software, y_train_software, y_test_software
        else:
            target, X_train, y_train, y_test = 'Platform', X_train_platform, y_train_platform, y_test_platform

    print(f"\nEvaluating {target} with {model_name}")

    # Define GridSearchCV for each model
    grid_search = GridSearchCV(model, param_grids[model_name], cv=3, scoring='f1_macro')

    # Fit the model
    grid_search.fit(X_train, y_train)

    # Predict and evaluate
    y_pred = grid_search.predict(X_test)
    print(f"Results for {target} with {model_name}:")
    print(classification_report(y_test, y_pred, zero_division=0))

    # Print best parameters for each model
    print(f"Best parameters found for {target} with {model_name}: {grid_search.best_params_}")


Training and evaluating Logistic Regression...

Evaluating Platform with Logistic Regression
Results for Platform with Logistic Regression:
                    precision    recall  f1-score   support

                         0.80      0.80      0.80         5
           Android       0.64      0.50      0.56        14
      Jio ChargeIT       0.00      0.00      0.00         1
               Web       0.89      0.67      0.76        12
               iOS       0.58      0.78      0.67         9
streaming services       0.40      0.33      0.36         6

          accuracy                           0.60        47
         macro avg       0.55      0.51      0.53        47
      weighted avg       0.66      0.60      0.62        47

Best parameters found for Platform with Logistic Regression: {'C': 10}
Training and evaluating Gradient Boosting Software...

Evaluating Software with Gradient Boosting Software
Results for Software with Gradient Boosting Software:
              precision  

In [276]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report
from imblearn.over_sampling import SMOTE

# Load the dataset
sampled_data = pd.read_csv('final_data.csv')

# Preprocessing: Fill NaN values and convert target columns to string
sampled_data['combined_text'] = sampled_data['combined_text'].fillna('').astype(str)

# Prepare target variables
y_domain = sampled_data['Domain [Healthcare, Ecommerce, Finance, Education, Entertainment, Media]'].fillna('').astype(str)
y_software = sampled_data['Software Application [frontent, backend, mobile, desktop, cloud, gaming]'].fillna('').astype(str)
y_platform = sampled_data['Platform [AndroidTV,  ATV, H5TV, iOS, Android,Web, Jio STB, Jio Cinema, Jio ChargeIT, Celia, All Platforms, SDK, JioTV, Stage, Spike, Playstore, Fanverse, CTV, VOOT]'].fillna('').astype(str)

# Preprocessing and feature extraction
vectorizer = TfidfVectorizer()
X = vectorizer.fit_transform(sampled_data['combined_text'])

# Split the data into training and testing sets
X_train, X_test, y_train_domain, y_test_domain = train_test_split(X, y_domain, test_size=0.2, random_state=42)
X_train, X_test, y_train_software, y_test_software = train_test_split(X, y_software, test_size=0.2, random_state=42)
X_train, X_test, y_train_platform, y_test_platform = train_test_split(X, y_platform, test_size=0.2, random_state=42)

smote = SMOTE(random_state=42, k_neighbors=1)

# Function to apply SMOTE and check the number of samples
def apply_smote(X_train, y_train):
    if len(y_train.value_counts()) < 2:  # Check if there are fewer than 2 classes
        print("Not enough classes for SMOTE.")
        return X_train, y_train  # Return original if not enough classes
    return smote.fit_resample(X_train, y_train)

# Resample training set for each target variable
X_train_domain, y_train_domain = apply_smote(X_train, y_train_domain)
X_train_software, y_train_software = apply_smote(X_train, y_train_software)
X_train_platform, y_train_platform = apply_smote(X_train, y_train_platform)

# Define the model dictionary
models = {
    'Logistic Regression': LogisticRegression(random_state=42, max_iter=1000, class_weight='balanced'),  # For Domain
    'Gradient Boosting Software': GradientBoostingClassifier(random_state=42),  # For Software
    'Gradient Boosting Platform': GradientBoostingClassifier(random_state=42)  # For Platform
}

# Define the parameter grid for hyperparameter tuning for each model
param_grids = {
    'Logistic Regression': {'C': [0.1, 1, 10]},
    'Gradient Boosting Software': {'n_estimators': [50, 100], 'max_depth': [3, 5, 10]},
    'Gradient Boosting Platform': {'n_estimators': [50, 100], 'max_depth': [3, 5, 10]}
}

# Loop through the models and perform training, tuning, and evaluation
for model_name, model in models.items():
    print(f"Training and evaluating {model_name}...")

    # Determine the target variable
    if "Logistic Regression" in model_name:
        target, X_train, y_train, y_test = 'Domain', X_train_domain, y_train_domain, y_test_domain
    elif "Software" in model_name:
        target, X_train, y_train, y_test = 'Software', X_train_software, y_train_software, y_test_software
    else:  # For Gradient Boosting Platform
        target, X_train, y_train, y_test = 'Platform', X_train_platform, y_train_platform, y_test_platform

    print(f"\nEvaluating {target} with {model_name}")

    # Define GridSearchCV for each model
    grid_search = GridSearchCV(model, param_grids[model_name], cv=3, scoring='f1_macro')

    # Fit the model
    grid_search.fit(X_train, y_train)

    # Predict and evaluate
    y_pred = grid_search.predict(X_test)
    print(f"Results for {target} with {model_name}:")
    print(classification_report(y_test, y_pred, zero_division=0))

    # Print best parameters for each model
    print(f"Best parameters found for {target} with {model_name}: {grid_search.best_params_}")



Training and evaluating Logistic Regression...

Evaluating Domain with Logistic Regression
Results for Domain with Logistic Regression:
               precision    recall  f1-score   support

   E-commerce       0.67      0.57      0.62         7
    Education       1.00      0.80      0.89        10
Entertainment       1.00      0.55      0.71        11
      Finance       1.00      1.00      1.00        11
        Media       0.44      0.88      0.58         8

     accuracy                           0.77        47
    macro avg       0.82      0.76      0.76        47
 weighted avg       0.85      0.77      0.78        47

Best parameters found for Domain with Logistic Regression: {'C': 0.1}
Training and evaluating Gradient Boosting Software...

Evaluating Software with Gradient Boosting Software
Results for Software with Gradient Boosting Software:
              precision    recall  f1-score   support

     backend       0.55      0.67      0.60         9
       cloud       1.00   

In [278]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.metrics import classification_report
from imblearn.over_sampling import SMOTE

# Load the dataset
sampled_data = pd.read_csv('final_data.csv')

# Preprocessing: Fill NaN values and convert target columns to string
sampled_data['combined_text'] = sampled_data['combined_text'].fillna('').astype(str)

# Prepare target variables
y_domain = sampled_data['Domain [Healthcare, Ecommerce, Finance, Education, Entertainment, Media]'].fillna('').astype(str)
y_software = sampled_data['Software Application [frontent, backend, mobile, desktop, cloud, gaming]'].fillna('').astype(str)
y_platform = sampled_data['Platform [AndroidTV,  ATV, H5TV, iOS, Android,Web, Jio STB, Jio Cinema, Jio ChargeIT, Celia, All Platforms, SDK, JioTV, Stage, Spike, Playstore, Fanverse, CTV, VOOT]'].fillna('').astype(str)

# Preprocessing and feature extraction
vectorizer = TfidfVectorizer()
X = vectorizer.fit_transform(sampled_data['combined_text'])

# Split the data into training and testing sets
X_train, X_test, y_train_domain, y_test_domain = train_test_split(X, y_domain, test_size=0.2, random_state=42)
X_train, X_test, y_train_software, y_test_software = train_test_split(X, y_software, test_size=0.2, random_state=42)
X_train, X_test, y_train_platform, y_test_platform = train_test_split(X, y_platform, test_size=0.2, random_state=42)

smote = SMOTE(random_state=42, k_neighbors=1)

# Function to apply SMOTE and check the number of samples
def apply_smote(X_train, y_train):
    if len(y_train.value_counts()) < 2:  # Check if there are fewer than 2 classes
        print("Not enough classes for SMOTE.")
        return X_train, y_train  # Return original if not enough classes
    return smote.fit_resample(X_train, y_train)

# Resample training set for each target variable
X_train_domain, y_train_domain = apply_smote(X_train, y_train_domain)
X_train_software, y_train_software = apply_smote(X_train, y_train_software)
X_train_platform, y_train_platform = apply_smote(X_train, y_train_platform)

# Define the model and parameter grid for hyperparameter tuning
model = GradientBoostingClassifier(random_state=42)
param_grid = {'n_estimators': [50, 100], 'max_depth': [3, 5, 10]}

# Function to train and evaluate model
def train_and_evaluate(target_name, X_train, y_train, X_test, y_test):
    print(f"Training and evaluating Gradient Boosting for {target_name}...")
    grid_search = GridSearchCV(model, param_grid, cv=3, scoring='f1_macro')
    
    # Fit the model
    grid_search.fit(X_train, y_train)
    
    # Predict and evaluate
    y_pred = grid_search.predict(X_test)
    print(f"Results for {target_name}:")
    print(classification_report(y_test, y_pred, zero_division=0))
    
    # Print best parameters for each model
    print(f"Best parameters found for {target_name}: {grid_search.best_params_}")

# Train and evaluate for all three targets
train_and_evaluate('Domain', X_train_domain, y_train_domain, X_test, y_test_domain)
train_and_evaluate('Software', X_train_software, y_train_software, X_test, y_test_software)
train_and_evaluate('Platform', X_train_platform, y_train_platform, X_test, y_test_platform)


Training and evaluating Gradient Boosting for Domain...
Results for Domain:
               precision    recall  f1-score   support

   E-commerce       0.67      0.86      0.75         7
    Education       0.77      1.00      0.87        10
Entertainment       1.00      0.55      0.71        11
      Finance       1.00      0.91      0.95        11
        Media       0.67      0.75      0.71         8

     accuracy                           0.81        47
    macro avg       0.82      0.81      0.80        47
 weighted avg       0.84      0.81      0.80        47

Best parameters found for Domain: {'max_depth': 3, 'n_estimators': 100}
Training and evaluating Gradient Boosting for Software...
Results for Software:
              precision    recall  f1-score   support

     backend       0.55      0.67      0.60         9
       cloud       1.00      0.94      0.97        18
    frontend       0.43      0.43      0.43         7
      gaming       0.86      0.67      0.75         9
   